In [17]:
!pip install konlpy

In [18]:
!pip install sentence_transformers

특정 문서의 주요 정보를 이해하고자 할 때 키워드 추출을 통해서 입력 텍스트와 가장 관련이 있는 단어와 구문을 추출할 수 있습니다. 'Rake'나 'YAKE!'와 같은 키워드를 추출하는 데 사용할 수 있는 패키지가 이미 존재합니다. 그러나 이러한 모델은 일반적으로 텍스트의 통계적 특성에 기반하여 작동하며 의미적인 유사성에 대해서는 고려하지 않습니다. 의미적 유사성을 고려하기 위해서 여기서는 SBERT 임베딩을 활용하여 사용하기 쉬운 키워드 추출 알고리즘인 KeyBERT를 사용합니다.

# 1. 기본 KeyBERT

In [19]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

이 튜토리얼에서는 드론에 대한 한국어 문서를 사용합니다.
-> 여기서는 하루치의 질의문을 전부 가지고 설명해보도록 하자


In [20]:
# doc = """
#          드론 활용 범위도 점차 확대되고 있다. 최근에는 미세먼지 관리에 드론이 활용되고 있다.
#          서울시는 '미세먼지 계절관리제' 기간인 지난달부터 오는 3월까지 4개월간 드론에 측정장치를 달아 미세먼지 집중 관리를 실시하고 있다.
#          드론은 산업단지와 사업장 밀집지역을 날아다니며 미세먼지 배출 수치를 점검하고, 현장 모습을 영상으로 담는다.
#          영상을 통해 미세먼지 방지 시설을 제대로 가동하지 않는 업체와 무허가 시설에 대한 단속이 한층 수월해질 전망이다.
#          드론 활용에 가장 적극적인 소방청은 광범위하고 복합적인 재난 대응 차원에서 드론과 관련 전문인력 보강을 꾸준히 이어가고 있다.
#          지난해 말 기준 소방청이 보유한 드론은 총 304대, 드론 조종 자격증을 갖춘 소방대원의 경우 1,860명이다.
#          이 중 실기평가지도 자격증까지 갖춘 ‘드론 전문가’ 21명도 배치돼 있다.
#          소방청 관계자는 "소방드론은 재난현장에서 영상정보를 수집, 산악ㆍ수난 사고 시 인명수색·구조활동,
#          유독가스·폭발사고 시 대원안전 확보 등에 활용된다"며
#          "향후 화재진압, 인명구조 등에도 드론을 활용하기 위해 연구개발(R&D)을 하고 있다"고 말했다.
#       """

doc="""
청강
청강신청
레인보우시스템
전과는 어떻게 할 수 있나요
레인보우
입학에 대해 궁금한게 있어요
e-Ad 메뉴_e-Advisor 설명
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
독후감으로 학점 어떻게 채우나요?
레인보우시스템은 어떤 시스템인가요?
학식 메뉴 확인할래요
비대면수업 및 대면수업 일정 안내
등록금 납부 방법에 대해서 알려주세요
독후감으로 학점 어떻게 채우나요?
마이크로
마이크로
회원가입 방법좀알려주세요
마이크로소프트
이수 학점 확인하고 싶어요
수강정정
대학규정 및 학칙에 대해 알려주세요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
대학규정 및 학칙에 대해 알려주세요
재수강 과목
공학인증 재수강
재수강
전과 학점
학생증 신청
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 조식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
나의 수업(e클래스) 과제를 확인하고싶어요
나의 강의별 과제/시험 점수를 알고싶어요
e클래스를 사용하고싶어요
열람용 성적
증명서 발급 방법 알려주세요
취업계
전과는 어떻게 할 수 있나요
재수강
평점평균
이수체계도 확인하기
재입학 방법 알려주세요
이수 학점 확인하고 싶어요
예술대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀 연락처 알려주세요
전공필수
에스컬레이터
학적등록
졸업하기 위해 필요한 요건이 어떻게 되나요?
학생증 학적등록 안 하고 도서관 출입에 찍었는데 그러면 재발급 받아야 돼?
증명서 발급 방법 알려주세요
예술대학 교학지원팀 연락처 알려주세요
기숙사
부서명
e클래스를 사용하고싶어요
교내 연락처 검색하고 싶어
교내 연락처 검색하고 싶어
이수 학점 확인하고 싶어요
학사일정
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
졸업인정제(영어/한자)에 대해서 알려주세요
졸업하기 위해 필요한 요건이 어떻게 되나요?
휴학 신청하는법
휴학신청
편입생 학점인정을 안내해주세요
경영경제소프트웨어
편입생 학점인정을 안내해주세요
강의시간표 확인하고 싶어요
에스컬레이터 평일 운영
강의계획서를 확인하고 싶어요
수업 폐강
졸업하기 위해 필요한 요건이 어떻게 되나요?
에스컬레이터
서울캠 내일 석식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
강의계획서를 확인하고 싶어요
재학증명서
강의계획서를 확인하고 싶어요
예술대학 교학지원팀 연락처 알려주세요
cau세미나 신청방법을 알려주세요
단과대학 교학지원팀 연락처
레인보우
서울캠 내일 석식메뉴
레인봉
수강신청은 언제, 어떻게 하나요?
최대학점
다빈치캠퍼스 교학지원팀 전화번호 알려줘
강의평가는 언제 어떻게 진행하나요?
조기졸업
졸업하기 위해 필요한 요건이 어떻게 되나요?
다빈치캠 교학지원팀
중도 사물함
학점등록은 언제/어떻게 하나요
다빈치캠 학부
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
계절학기
서울 조식 메뉴
강의계획서
학식 메뉴 확인할래요
국가근로
강의계획서를 확인하고 싶어요
학생증 재발급
온라인 폭력예방교육이 뭐야?
법정교육에 대해 안내해주세요
중앙도서관 사물함
재수강 성적은 기본적으로 0.5점 깎이나요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
sw프로그램 지원
공결서
학식 메뉴 확인할래요
이수 학점 확인하고 싶어요
cau세미나 신청방법을 알려주세요
e클래스를 사용하고싶어요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
서울캠 내일 석식메뉴
학식 메뉴 확인할래요
서울캠 내일 중식메뉴
학생증 분싷
수강신청 정정
서울캠 내일 조식메뉴
서울캠 내일 학식메뉴
서울 조식 메뉴
학식 메뉴 확인할래요
독후감
이수 학점 확인하고 싶어요
이수구분신청
졸업하기 위해 필요한 요건이 어떻게 되나요?
도시계획부동산학과 졸업요건
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
e클래스를 사용하고싶어요
학식 메뉴 확인할래요
졸업요건
이수 학점 확인하고 싶어요
Cau.ac.kr
학사일정
기말고사
자유선택과목 인정이 뭔가요
자율선택과목은 교양에 포함되나요?
이수구분 신청서
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
휴학
서울캠 내일 석식메뉴
서울캠 내일 중식메뉴
서울캠 내일 조식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
선택교양과 자율선택은 달라?
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
졸업하기 위해 필요한 요건이 어떻게 되나요?
대학 및 학과(부)별 졸업이수학점 알려주세요
서울 중식 메뉴
학식 메뉴 확인할래요
월별 학사 일정이 궁금해요
융합전공
강의시간표 확인하고 싶어요
수료생이 뭐야
이수 학점 확인하고 싶어요
수료생
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
독후감
소프트웨어학부
소프트웨어
다빈치 내일 조식메뉴
다빈치캠퍼스 내일 학식메뉴
김효수
다빈치 내일 중식메뉴
교직원/교수님
교내 연락처 검색하고 싶어
김효수 교수
다빈치캠퍼스 내일 석식메뉴
다빈치캠퍼스 천원의아침밥 먹을래
다빈치캠 내일 학식메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
김효수 교수
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치 내일 중식메뉴
다빈치캠 내일 학식메뉴
학식 메뉴 확인할래요
현재 취득학점
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
졸업학점
다빈치 내일 조식메뉴
다빈치캠 내일 학식메뉴
학식 메뉴 확인할래요
계절학기
인건비
추가학기
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
강의마당
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치 중식 메뉴
다빈치 중식 메뉴
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
단과대학별 공지사항 있나요?
다빈치캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
ppt sw 설치
졸업을 위한 논문/시험/작품에 대해서 알려주세요
졸업 요건
독후감
강의평가는 언제 어떻게 진행하나요?
졸업 요건
신입생 장학금
서울 중식 메뉴
3월 6일 중식
장학금 정보가 궁금해요
서울 중식 메뉴
학식 메뉴 확인할래요
서울캠 내일 석식메뉴
졸업하기 위해 필요한 요건이 어떻게 되나요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
셔틀
연구실안전 법정교육이 뭔가요
등록금납부
졸업하기 위해 필요한 요건이 어떻게 되나요?
다빈치캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
셔틀버스 시간표와 노선을 알고 싶어요
경조사
졸업 가능여부(졸업사정)를 확인하고 싶어요
학점 등록
셔틀버스 시간표와 노선을 알고 싶어요
일취월장
전공심화는 무엇인가요?
전공인정
일취월장
예술대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀 연락처
부서별 연락처안내
재무회계팀 연락처 알려주세요
자퇴 신청은 어떻게 하나요
자퇴
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
도서관
통학버스
재수강
셔틀
한자
항자
셔틀버스 시간표와 노선을 알고 싶어요
전공심화과목
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
포탈
셔틀버스 시간표와 노선을 알고 싶어요
셔틀버스 시간표와 노선을 알고 싶어요
ppt 설치하는방법 알려줘
다빈치캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
ppt설치하고 싶은데 교육용 이메일좀 줘
달력
졸업예정증명서
월별 학사 일정이 궁금해요
경영학점
이수 학점 확인하고 싶어요
서울캠 내일 조식메뉴
서울캠 내일 학식메뉴
이수 학점 확인하고 싶어요
학점
졸업인정제(영어/한자)에 대해서 알려주세요
졸업하기 위해 필요한 요건이 어떻게 되나요?
외부장학 공지
장학금 정보가 궁금해요
서울 중식 메뉴
교직원/교수님
교내 연락처 검색하고 싶어
서울 조식 메뉴
학식 메뉴 확인할래요
졸업인정제(영어/한자)에 대해서 알려주세요
졸업기준
졸업인정제(영어/한자)에 대해서 알려주세요
졸업하기 위해 필요한 요건이 어떻게 되나요?
서울캠 내일 석식메뉴
졸업
학생증 교부는 언제되나요?
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학생증 발급
서울 중식 메뉴
학식 메뉴 확인할래요
재적증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
졸업
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
건축학부
예술대학 연습실
연습실
재학증명서
학점등록
단과대학별 공지사항 있나요?
졸업을 위한 논문, 시험, 작품에 대해서 알려주세요
서울캠 내일 조식메뉴
서울캠퍼스 학식 메뉴 확인할래요
졸업인정제(영어/한자)에 대해서 알려주세요
졸업하기 위해 필요한 요건이 어떻게 되나요?
계절학기
계절학기 수강신청은 언제/어떻게 하나요
서울캠 내일 석식메뉴
연체료 얼마야
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
셔틀
도서 대출 한도 안내
학점등록
도서 대출 안내
도서관 이용 방법 알려주세요
학식 메뉴 확인할래요
등록금납입내역 확인할래요
학생증 교부는 언제되나요?
등록금 납부 방법에 대해서 알려주세요
학생증신청
입학에 대해 궁금한게 있어요
이용Tip
외부장학
중앙대 장학금
신청 가능한 장학금이 있나요?
내 장학 금액이 궁금해
장학금 신청내역 조회
장학금 신청
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
104관 위치
810관
서울캠 내일 석식메뉴
중앙대 장학금
장학법인
장학금
캠퍼스맵(건물위치) 보여주세요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
기타 부득이한 사유
질병결석
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
학칙문의
해외협정대학 단기파견 프로그램 지원자격 알려주세요
대학규정 및 학칙에 대해 알려주세요
eclass 어플
구내서점
어떤 복지시설(편의시설)이 있나요?
성적증명서 발급 안내해주세요
조직도
처음화면
조직도
부서명
인문대 교학지원팀
생명공학대학 교학지원팀 연락처 알려주세요
졸업 관련 증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
인문대학 행정실
단과대학 교학지원팀 연락처
석차
대학원 교학지원팀 연락처
서울캠 내일 중식메뉴
교내 연락처 검색하고 싶어
서울캠퍼스 학식메뉴
식당 메뉴
교내 연락처 검색하고 싶어
졸업 관련 증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
청강
청갇
장학 수혜 내역
등록금납입내역 확인할래요
서울캠 내일 중식메뉴
등록금 납부 방법에 대해서 알려주세요
서울캠 내일 조식메뉴
서울캠 내일 학식메뉴
서울 조식 메뉴
학식 메뉴 확인할래요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
등록금 고지서가 조회가 안되요
나의 수업(e클래스) 과제를 확인하고싶어요
Eclass
찰리야 졸업하고 싶어
생명공학대학 학생회칙
찰리야 자퇴할래
대체과목인정은 어떻게 신청해?
대체과목인정
첨단강의실
수업 인정
강의 녹화
학생증/신분증 사진 변경 방법 알아보기
포탈 사진 변경 어떻게 하나요
다빈치 중식 메뉴
졸업하기 위해 필요한 요건이 어떻게 되나요?
졸업정보
다빈치캠퍼스 천원의아침밥 먹을래
학생증/신분증 사진 변경 방법 알아보기
서울캠퍼스 천원의아침밥 먹을래
다빈치 조식 메뉴
다빈치 석식 메뉴
다빈치 중식 메뉴
학식 메뉴 확인할래요
서울캠퍼스 천원의아침밥 먹을래
서울캠 내일 조식메뉴
서울캠 내일 학식메뉴
전자출결시스템
서울 중식 메뉴
마일리지 적립
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
미래설계장학금
학식 메뉴 확인할래요
포탈 사진 변경
학생증/신분증 사진 변경 어떻게 하나요
입학에 대해 궁금한게 있어요
중앙나래장학금
공과대학 교학지원팀 연락처 알려주세요
대학별 교학지원팀 연락처 알려주세요
다빈치 조식 메뉴
다빈치캠퍼스 학식메뉴
출석체크 확인부탁해
학생식당 위치
분할납부
복지장학금
다빈치캠 내일 학식메뉴
학식 메뉴 확인할래요
재학생 장학금 안내
역량강화장학금
부속기관장학금
재학생 장학금
장학금 정보가 궁금해요
공과대학 교학지원팀 연락처 알려주세요
대학별 교학지원팀 연락처 알려주세요
선수과목학점인정서
졸업증명서
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
도서관 이용 방법 알려주세요
재학증명서
융합전공
다빈치캠퍼스 주차장 이용 안내
주차장 이용에 대해 안내해주세요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
로고
레인보우시스템은 어떤 시스템인가요?
출석 어떻게 하나요
내가 신청한 비교과 프로그램 일정을 알려주세요
이수 학점 확인하고 싶어요
비교과프로그램 완료
서울캠 내일 석식메뉴
학생증/신분증 발급내역 조회할 수 있나요
서울캠 내일 조식메뉴
서울캠 내일 학식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
서울캠퍼스 천원의아침밥 먹을래
아침식사
수강신청 가능학점은 몇 학점 인가요?
내가 신청한 비교과 프로그램 일정을 알려주세요
비교과 프로그램 알려주세요
비교과 학습체계(LEAP)는 무엇인가요?
도서관 이용 방법 알려주세요
강신청은 언제, 어떻게 하나요?
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
비교과프로그램 후기작성
서울 중식 메뉴
수강신청 학점 이월이 가능한가요?
학식 메뉴 확인할래요
후기작성
마일리지
커피점
제과점
수강가능학점
교내 편의시설 안내
다빈치캠퍼스 석식 메뉴
무인택배
한학기만 휴학 가능해?
공통교양 재수강 불가
재적증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
다빈치 중식 메뉴
학식 메뉴 확인할래요
F 재수강 불가
성적증명서
matlab
재수강 불가
학점등록
matlab
독후감
독서
학점등록
교육학과 학점제한
서울캠 내일 중식메뉴
내일 점심메뉴
선수과목 점수는 60이에요?
서울 중식 메뉴
학술정보원
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
추가학기
서울캠 내일 조식메뉴
서울캠 내일 학식메뉴
경영경제대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀 연락처
서울 조식 메뉴
학점등록
학식 메뉴 확인할래요
다빈치캠퍼스 내일 석식메뉴
서울캠 내일 석식메뉴
gds
졸업관련정보이수
다빈치캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
다빈치 내일 중식메뉴
다빈치캠 내일 학식메뉴
학식 메뉴 확인할래요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
강신청은 언제, 어떻게 하나요?
다빈치 내일 중식메뉴
다빈치캠퍼스 내일 학식메뉴
다빈치 중식 메뉴
학식 메뉴 확인할래요
수강신청은 언제, 어떻게 하나요?
석차
미디어커뮤니케이션 학부 과 사무실 연락처
서울캠퍼스 학식 메뉴 확인할래요
재학증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
핵심교양
서울캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
월별 학사 일정이 궁금해요
재학증명서 발급 안내해주세요
다빈치 중식 메뉴
월별 학사 일정이 궁금해요
증명서 발급 방법 알려주세요
서울캠퍼스 주차장 이용 안내
학식 메뉴 확인할래요
주차장 이용에 대해 안내해주세요
졸업하기 위해 필요한 요건이 어떻게 되나요?
서울캠 내일 중식메뉴
이수 학점 확인하고 싶어요
서울캠 내일 조식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
졸업정보
핵심 교양은 종류별로 하나씩 다 들어야 졸업이 인정되는거야?
다빈치 내일 조식메뉴
포트폴리오 수업
연구실안전 법정교육이 뭔가요
전공평점평균 어디서 알수 있나요?
다빈치 중식 메뉴
전공학점 평균 어떻게 알아
대학별 교학지원팀 연락처 알려주세요
다빈치 조식 메뉴
서울 조식 메뉴
다빈치캠 내일 학식메뉴
학식 메뉴 확인할래요
학점등록은 언제, 어떻게 하나요?
등록금 추가학기
강의실 대여할 수 있나요
등록금 납부 방법에 대해서 알려주세요
서울캠 내일 석식메뉴
경영대학원
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
강의 계획서
다빈치 내일 중식메뉴
서울 중식 메뉴
서울 조식 메뉴
서울캠퍼스 학식 메뉴
다빈치 내일 조식메뉴
다빈치캠퍼스 내일 학식메뉴
다빈치 조식 메뉴
다빈치캠퍼스 내일 석식메뉴
다빈치 내일 중식메뉴
다빈치캠 내일 학식메뉴
학식 메뉴 확인할래요
서울캠 내일 조식메뉴
서울캠 내일 학식메뉴
서울캠퍼스 천원의아침밥 먹을래
서울캠 내일 중식메뉴
학식 메뉴 확인할래요
서울캠 내일 학식메뉴
서울 중식 메뉴
기숙사 헬스장
학식 메뉴 확인할래요
휘트니스실
인턴보고서 서식
학생증 재발급
경영경제대학
서울 석식 메뉴
학식 메뉴 확인할래요
단과대학별 공지사항 있나요?
졸업시험
예술대학
졸업시험
인턴보고서 서시
다빈치캠퍼스 내일 석식메뉴
다빈치 내일 중식메뉴
다빈치캠 내일 학식메뉴
학식 메뉴 확인할래요
미디어실
수시 경쟁률
서울 중식 메뉴
선수과목학점인정서 어디에 받을 수 있나요?
학식 메뉴 확인할래요
이수 학점 확인하고 싶어요
졸업하기 위해 필요한 요건이 어떻게 되나요?
선수과목인전서
복사실
분식점
어떤 복지시설(편의시설)이 있나요?
어학연수
서울캠 내일 석식메뉴
굿즈
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
중식
e클래스를 사용하고싶어요
대학원 어학시험 신청
대학원 어학시험
대학원 어학
재학증명서 발급 안내해주세요
등록금납입확인서
증명서 발급 방법 알려주세요
포트폴리오 이용방법에 대해 안내해주세요
졸업일
포트폴리오 이용방법에 대해 안내해주세요
계절학기
졸업일
어학연수
어학연수
단과대학 교학지원팀
대학원어헉시험
서울캠 내일 석식메뉴
어학시험
시험신청
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
다빈치 중식 메뉴
다빈치캠 내일 학식메뉴
학식 메뉴 확인할래요
도서관 이용 방법 알려주세요
중앙대학교 모바일 앱을 알고 싶어요
열람실 좌석배정
도서관
수강신청은 언제, 어떻게 하나요?
인문대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀 연락처 알려주세요
전공 점수
필요학점\
학점
adobe 학생할인
프린터
이용Tip
다빈치 조식 메뉴
학식 메뉴 확인할래요
도서관 이용 방법 알려주세요
대외활동
증명서 발급방법과 수수료에 대해 알려주세요
증명서
재수강
연구실안전 법정교육이 뭔가요
강의계획서를 확인하고 싶어요
모바일학생증 다운로드 방법
모바일학생증 안내해주세요
서울 석식 메뉴
학식 메뉴 확인할래요
순수한 학생증
스마트학생증 재발급
학생증 신청
전과 모집요강
연체료 졸업
연체 졸업
E ID
교내토익
서울도서관 운영시간
도서관 이용시간
헬스장
스마트학생증 신규발급
학생증
모바일 학생증/신분증 분실등록 및 기변신청은 어떻게 하나요
융합전공
내정보 수정
생활관
수령장소확인
학생증/신분증 발급내역 조회할 수 있나요
다빈치 내일 중식메뉴
다빈치캠 내일 학식메뉴
장학금 정보가 궁금해요
학식 메뉴 확인할래요
편의시설 안내
기숙사에서 음식물쓰레기 어디에 버려야해?
봅스트홀 샤워실 운영시간
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
샤워실
다빈치캠 내일 학식메뉴
학식 메뉴 확인할래요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
신입생 장학금
장학금
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
도시계획부동산학과
학식
특례편입
캠퍼스맵(건물위치) 보여주세요
생활관(기숙사)에 대해 알려줘
학생증 재발급
교환
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
휴학 환불
다빈치러닝이 무엇인가요
수업시간표 알려줘
강의시간표 확인하고 싶어요
절대평가 기준
일취월장 신청
일취월장
수강신청 정정 일정
수강신청은 언제, 어떻게 하나요?
경영학부 행정실
수강신청 정정기간
e상담센터 상담 신청
수업 출석체크를 못했는데 어떡하지?
학생증 학적등록
학생증 학저그
민원이 있으면
학원론
졸업예정 수료
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
수료 단기파견
고시반에 대해 알고싶어요
cpa
동아리
ㄷㅇ아리
사단
변리사
e-Ad 메뉴_e-Advisor 설명
상담 일정 확인하고 싶어요
동아리
고시반에 대해 알고싶어요
국가시험장학금
재학생 장학금 안내
미래설계장학금
동아리
재학생장학금
신청 가능한 장학금이 있나요?
장학금 신청내역 조회
내 장학 금액이 궁금해
가족장학금
재학생 장학금은 어떤게 있나요?
장학금 신청내역 조회
신청 가능한 장학금이 있나요?
다빈치캠퍼스 천원의아침밥 먹을래
신청 장학 확인
학식 메뉴 확인할래요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
장학금 신청내역 조회
신청 가능한 장학금이 있나요?
장학금 신청내역 조회
신청 가능한 장학금이 있나요
서울 중식 메뉴
학식
교내장학 신청
민원
이수 학점 확인하고 싶어요
국가자격증에 대해 알려주세요
강의시간표 확인하고 싶어요
고시반에 대해 알고싶어요
서울도서관 운영시간
도서관 운영시간 안내
도서관 이용 방법 알려주세요
해외협정대학 단기파견 프로그램 지원자격 알려주세요
고시반에 대해 알고싶어요
모바일학생증 기변신청 방법
모바일학생증 발급방법
모바일 학생증/신분증 분실등록 및 기변신청은 어떻게 하나요
해외교류프로그램 장학금에 대해 알고싶어요
실감미디어
도서관 이용 방법 알려주세요
고시반에 대해 알고싶어요
교육혁신원
에스컬레이터
고시반에 대해 알고싶어요
스마트학생증 신규발급
이수 학점 확인하고 싶어요
순수학생증 만드는 기간
필수로 이수해야하는 교양과목이 무엇인가요?
mach교양과목이 무엇인가요
다전공
졸업인정제(영어/한자)에 대해서 알려주세요
다빈치 내일 조식메뉴
다빈치 내일 중식메뉴
졸업요건
다빈치캠 내일 학식메뉴
학식 메뉴 확인할래요
엑셀
사회과학대학 교학지원팀 연락처 알려주세요
교내 연락처 검색하고 싶어
졸업요건
다빈치 내일 중식메뉴
다빈치캠퍼스 내일 학식메뉴
엑셀
다빈치 중식 메뉴
학식 메뉴 확인할래요
서울캠 내일 조식메뉴
서울캠 내일 학식메뉴
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
엑셀
마이크로소프트
서울캠 내일 석식메뉴
서울캠 내일 학식메뉴
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
강의시간표 확인하고 싶어요
강의시간표 확인하고 싶어요
모바일학생증 기변신청 방법
모바일 학생증에서 왼쪽으로 슬라이드 하는데 큐알코드 가 나타나지 않아요
큐알코드가 안생경ㅛ
재학생장학금
수강신청
중앙사랑장학금
중앙사랑
모바일 학생증에서 출석체크가 안되는데 어떡하죠?
중앙사랑 지급일
강신청은 언제, 어떻게 하나요?
연구실 안전
수강신청
졸업인정제(영어/한자)에 대해서 알려주세요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
학생증/신분증 사진 변경 방법 알아보기
전자출결 사진
추가학기 등록금
졸업요건
폐강 기준
폐강
서울캠 내일 조식메뉴
서울캠 내일 학식메뉴
서울캠퍼스 천원의아침밥 먹을래
천원
광고홍보학과
1학점 인정
1학점 교양
독후감
수강정정
의학부 사무실
자퇴
아이림스
학생증 배송 받았는데 또 뭐 해야되는 절차가 있어?
e클래스를 사용하고싶어요
장학내역
출석현황
서울캠 내일 중식메뉴
스마트학생증 신규발급
출석인정제 신청하고 싶어요
학생증
서울캠 내일 석식메뉴
등록금
강의계획서를 확인하고 싶어요
멘티
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
재학증명서
강의계획서를 확인하고 싶어요
화학과
출석인정제 신청하고 싶어요
이수구분변경신청
융합전공
e클래스를 사용하고싶어요
증명서 발급 방법 알려주세요
건강센터 연락처
중앙사랑장학금
건강검진에 대해 알려주세요
중앙사랑
음식점 어디있어?
나의 강의교재 확인하기
중앙대학교 강의마당
나의 수업(e클래스) 강의자료를 확인하고싶어요
도서관 이용 방법 알려주세요
패스트푸드
University Club
학점이월제
어떤 복지시설(편의시설)이 있나요?
e클래스를 사용하고싶어요
중앙대병원 혜택
도서관 이용 방법 알려주세요
나의 수업(e클래스) 강의자료를 확인하고싶어요
e클래스를 사용하고싶어요
도서관서비스
패스트푸드
분식점
e클래스를 사용하고싶어요
기념품판매점
어떤 복지시설(편의시설)이 있나요?
음식점
수강신청은 언제, 어떻게 하나요?
서울캠 내일 조식메뉴
서울캠 내일 학식메뉴
서울 조식 메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
다빈치 석식 메뉴
다빈치 학식 메뉴 알려줘
다빈치캠퍼스 내일 중식메뉴
다빈치캠퍼스 학식 메뉴 확인할래요
소프트웨어 다운로드는 어디에서 받나요?
등록
워드
졸업사정현황
M.S word(docx)
졸업하기 위해 필요한 요건이 어떻게 되나요?
추가학기 학점등록
편입생 5학년 등록금
편입생 추가학기 등록금
계절학기
이수 학점 확인하고 싶어요
학생증 재발급
타과 수강
강의시간표 확인하고 싶어요
정정기간에 타학과 전용 교양강의 수강신청해도 돼?
e클래스를 사용하고싶어요
학점등록은 언제, 어떻게 하나요?
9학점 미만
강신청은 언제, 어떻게 하나요?
포탈 계정 변경
학점등록
아이디 변경 가능할까요
이용Tip
학점등록은 언제, 어떻게 하나요?
9학점 미만
학생증 재발급
학생증/신분증 사진 변경 어떻게 하나요
엑셀
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
전자자료검색
출결 정정
도서관 서비스
학사 가이드
e클래스를 사용하고싶어요
KMOOC
출결현황 조회할래요
다빈치 내일 중식메뉴
다빈치 내일 조식메뉴
다빈치캠 내일 학식메뉴
중앙대학교 지도
학식 메뉴 확인할래요
전화번호 뭐야
ATM
은행 운영시간
중앙사랑장학금
재학생 장학금
장학금 정보가 궁금해요
선택교양
서울도서관 운영시간
도서관 운영시간
졸업요건
커피점
구두수선소 어디있어?
봉사활동을 학점으로 인정받을 수 있나요
교내 편의시설 안내
eid 기기변경
1학점
학생식당 위치
음식점 어디있어?
제과점 어디있어?
수강신청
과잠
서울캠 내일 석식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
베이글
수강신청
다빈치 내일 중식메뉴
다빈치캠퍼스 내일 학식메뉴
과잠
다빈치 내일 조식메뉴
다빈치캠퍼스 내일 학식메뉴
교내 편의시설 안내
다빈치 조식 메뉴
학식 메뉴 확인할래요
기념품샵 알려줘
교복
와이파이 사용방법이 궁금해요
강의계획서를 확인하고 싶어요
학술자료를 검색하고 싶어요
학술정보원(도서관) 모바일 앱이 있나요
와이파이 사용방법이 궁금해요
다빈치캠버스 801관 와이파이
다빈치캠퍼스 주차장 이용 안내
주차
spss
연체료 얼마야
도서 대출 한도 안내
이수 학점 확인하고 싶어요
핵교
서울캠 내일 석식메뉴
e클래스를 사용하고싶어요
다빈치캠버스 801관 와이파이
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
다빈치 내일 중식메뉴
다빈치캠 내일 학식메뉴
학식 메뉴 확인할래요
스마트학생증/신분증 신규발급 신청방법 알려주세요
동일대체과목이란
다빈치캠버스 와이파이
e클래스를 사용하고싶어요
공대 팀플실
서울캠퍼스 스터디룸 예약
팀플실
e클래스를 사용하고싶어요
자퇴
서울캠 내일 석식메뉴
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
여름방학
내정보수정
다빈치 내일 중식메뉴
다빈치캠 내일 학식메뉴
학식 메뉴 확인할래요
월별 학사 일정이 궁금해요
튜터 신청
서울 중식 메뉴
다빈치 튜터
등록금 납부 방법에 대해서 알려주세요
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
도서 대출 안내
체육대학 홈페이지
서울도서관 운영시간
도서관 운영시간 안내
다른 학교(타대학) 도서관 이용이 가능한가요
도서관
지도교수
아이림스
학교 지도
서울 석식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
등록금 금액 확인하고 싶어요
서울 석식 메뉴
학식 메뉴 확인할래요
사범대학 안내해주세요
교직팀 홈페이지
등록금 납부 방법에 대해서 알려주세요
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
재학생장학금
재적증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
외국인장학금
재학생 장학금
학생지원팀 연락처
장학금 정보가 궁금해요
서울 석식 메뉴
학식 메뉴 확인할래요
교할ㅈ
교학지원팀 운
공과대학 교학지원팀 연락처 알려주세요
교내 연락처 검색하고 싶어
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
예술대학 교학지원팀 연락처 알려주세요
대학별 교학지원팀 연락처 알려주세요
패션 전공 학점 요구 사항
패션학과 3학년 편입학 졸업 요건
2021계절학기
2022계절학기
졸업인정제(영어/한자)에 대해서 알려주세요
졸업하기 위해 필요한 요건이 어떻게 되나요?
모바일 학생증/신분증 분실등록 및 기변신청은 어떻게 하나요
6층 피시실 예약
출석
어떤 경우에 출석인정이 가능한가요
출석
츌석
졸업요건
이수 학점 확인하고 싶어요
서울 석식 메뉴
학식 메뉴 확인할래요
보관성적 조회
서울 석식 메뉴
학식 메뉴 확인할래요
학점
허
학사일정
올해 학사일정 어디서 확인할 수 있어?
서울 석식 메뉴
학식 메뉴 확인할래요
재수강
해외건설 ppp
서울 석식 메뉴
학식 메뉴 확인할래요
Wifi
Pc버전
강의평가 점수
F 재수강
강의평가
강의평가 결과
와이파이 비밀번호
F
봉사활동을 학점으로 인정받을 수 있나요
봉사활동을 학점으로 인정받을 수 있나요
미디어커뮤니케이션
사회과학대학
강의평가
F
중앙사랑 증서환불
다빈치 석식 메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
증서환불
F
서울도서관 운영시간
도서관 운영시간 안내
서울 석식 메뉴
도서관 이용 방법 알려주세요
학식 메뉴 확인할래요
한자 제출 서류
졸업하기 위해 필요한 요건이 어떻게 되나요?
전과학점
e클래스를 사용하고싶어요
등록금납입내역 확인할래요
이전 등록금 고지서
아이림스
전과는 어떻게 할 수 있나요
서울 석식 메뉴
학식 메뉴 확인할래요
학사지원관
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
재학증명서 발급 안내해주세요
스마트학생증
증명서 발급 방법 알려주세요
학생증 교부는 언제되나요?
서울캠퍼스 천원의아침밥 먹을래
학생증 발급
학식 메뉴 확인할래요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
졸업하기 위해 필요한 요건이 어떻게 되나요?
학식 메뉴 확인할래요
전과는 어떻게 할 수 있나요
성적증명서
패션학과 3학년 편입학 졸업 학점 요구
동아리
서울 석식 메뉴
법학 도서관
학식 메뉴 확인할래요
서울캠퍼스 도서관
서울캠퍼스 도서관
중앙대 학술정보원
패션학과 3학년 편입생 졸업 요건
융합전공
융합전공 연락처
금융ai
금융 ai 연락처
다빈치 석식 메뉴
서울캠 내일 조식메뉴
학식 메뉴 확인할래요
서울캠 내일 학식메뉴
서울 조식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
나의 지도교수님은 누구인가요
중앙사랑장학금
학식
중앙사랑
나의 지도교수님은 누구인가요
서울 석식 메뉴
한자
인사팀
학식 메뉴 확인할래요
패션학과 졸업 요건
인사팀
교환학생
교환학생 하려면 어떻게 해야돼
중앙사랑장학금
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
부서별 연락처안내
인사팀
전과
교내 연락처 검색하고 싶어
중앙사랑장학금
중앙사랑
등록금 납부시기
캠퍼스맵(건물 위치) 보여주세요
자퇴
학교 건뭉
e-Ad 메뉴_e-Advisor 설명
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
포탈
강의계획서를 확인하고 싶어요
서울 석식 메뉴
학식 메뉴 확인할래요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
소프트웨어 다운로드는 어디에서 받나요?
어떤 복지시설(편의시설)이 있나요?
계절학기 수강신청은 언제, 어떻게 하나요?
e-Ad 메뉴_e-Advisor 설명
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
도서관 이용 방법 알려주세요
도서관 이용 방법 알려주세요
광고홍보학과
계절학기
계절학기 고지서는 어떻게 확인하나요
한글 설치
서울 석식 메뉴
학식 메뉴 확인할래요
패스트푸드
어떤 복지시설(편의시설)이 있나요?
서울 석식 메뉴
학식 메뉴 확인할래요
블루미르홀 308관 식당은 몇층이야?
서울 석식 메뉴
학식 메뉴 확인할래요
e-Ad 메뉴_e-Advisor 설명
다빈치 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
이수 학점 확인하고 싶어요
어디서 카메라를 빌릴 수 있습니까?
기념품
학사일정
다빈치 석식 메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
이수 학점 확인하고 싶어요
아이림스
빈강의실
입학에 대해 궁금한게 있어요
수강신청은 언제, 어떻게 하나요?
서울 중식 메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
융합전공
학생지원팀 전화번호
서울 석식 메뉴
서울캠퍼스 천원의아침밥 먹을래
학식 어디서 먹어?
현수막을 달고 싶어
수강신청 학점 이월이 가능한가요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
아이리스
서울 석식 메뉴
학식 메뉴 확인할래요
등록 기간
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
교내 와이파이 이용 방법 알려줘
서울 석식 메뉴
학식메뉴
재학증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
출석인정신청
장애학생지원센터 홈페이지
야간 전형이 있나요?
포탈
중앙대 야간이 있나요?
내 수업 시간표
수업시간표 알려줘
화학과
서울 석식 메뉴
논문자격제출
학식 메뉴 확인할래요
강의시간표 확인하고 싶어요
이수 학점 확인하고 싶어요
도서관 프린터
재입학
계절학기 수강신청은 언제/어떻게 하나요
서울캠 내일 석식메뉴
다빈치 석식 메뉴
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
재학증명서 발급 안내해주세요
졸업요건
증명서 발급 방법 알려주세요
등록금 납부
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
서울캠퍼스 학식 메뉴 확인할래요
교내 연락처 검색하고 싶어
다빈치 조식 메뉴
다빈치 학식메뉴 알려줘
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
다빈치캠 내일 학식메뉴
다빈치도서관 운영시간
다빈치도서관 운영시간
조식
다빈치도서관 운영시간
다빈치도서관 운영시간
다빈치도서관 운영시간
도서관 운영시간 안내
다빈치 내일 조식메뉴
다빈치캠 내일 학식메뉴
도서관 이용 방법 알려주세요
천원의빵
카우잇츠 베이글
학생식당 위치
베이글
휴학 몇년
학생증 재발급
휴학 규정
서울캠 내일 석식메뉴
서울캠 내일 석식메뉴
서울캠 내일 중식메뉴
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
서울캠퍼스 학식 메뉴 확인할래요
예비군 전입신고
서울 석식 메뉴
등록금납입내역 확인할래요
학식 메뉴 확인할래요
등록금 금액(등록금 고지서) 확인하고 싶어요
강의계획서를 확인하고 싶어요
입학 장학금 고지서
신입생 장학금은 어떤게 있나요
학식
등록금납입내역 확인할래요
등록금고지서
이수구분
교내셔틀
등록금 금액(등록금 고지서) 확인하고 싶어요
장학금 고지내역
서울 석식 메뉴
학식 메뉴 확인할래요
게시판
입학처 큐앤에이 게시판
입학에 대해 궁금한게 있어요
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
나의 수업(e클래스) 강의자료를 확인하고싶어요
서울 석식 메뉴
e클래스
학식 메뉴 확인할래요
공과대학 교학지원팀 연락처 알려주세요
전자교탁
전자교탁 사용메뉴얼
교내 연락처 검색하고 싶어
성적표 출력
강의시간표 확인하고 싶어요
졸업 영어
상담연결
학점평균
융합전공
일반대학원
졸업 관련 증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
학생지원팀
졸업 관련 증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
졸업 관련 증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
셔틀 요금
서울 중식 메뉴
학식 메뉴 확인할래요
경제학과사무소 전화번호 알려줘
수강한 과목의 이수 구분을 변경하고 싶어요
강신청은 언제, 어떻게 하나요?
전과
이수구분
셔틀비용
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
도서관
봉사활동을 학점으로 인정받을 수 있나요
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
이수구분
학식 메뉴 확인할래요
다빈치 석식 메뉴
학식 메뉴 확인할래요
서점
다빈치 조식 메뉴
학식 메뉴 확인할래요
재학증명서
사과대 과사무실
과사무실
공대 지원처
교과코드
문화콘텐츠
학식 위치
한자능력검정시험 증명서
학생증 재발급
한자증명서 양식
한자증명서
교내셔틀
교과코드
서울 석식 메뉴
학식 메뉴 확인할래요
학생식당 위치
학식 메뉴 확인할래요
셔틀버스
단과대학 교학지원팀 연락처
학점등록
서울 석식 메뉴
학식 메뉴 확인할래요
365
학생증 신청은 어떻게 돼?
부충장
다빈치 석식 메뉴
학식 메뉴 확인할래요
다빈치 석식 메뉴
학식 메뉴 확인할래요
소프트웨어학부 연락처
word
컴공 교학 연락처
기술고시
ms-word
최종등록
석식
서울 석식 메뉴
학식
학생지원팀 운영시간
소프트웨어학부 연락처
학점등록
소프트웨어 학사 연락처
등록기감
학생지원팀 연락처
스마트학생증
융합전공
한자
교내 부서(팀)별 연락처 알려주세요
서울 석식 메뉴
학식
서울 석식 메뉴
학식 메뉴 확인할래요
사범대학 교학지원팀 연락처 알려주세요
수강신청
교내 연락처 검색하고 싶어
메일
교내 연락처 검색하고 싶어
학생지원팀 연락처 알려줘
예술대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀 전화번호
중앙대학교 학생지원팀 전화번호좀 알려줘
이수 학점 확인하고 싶어요
서울 석식 메뉴
학식 메뉴 확인할래요
스마트학생증 재발급
학생증 교부는 언제되나요?
학생증/신분증 사진 변경 어떻게 하나요
서울 석식 메뉴
학식 메뉴 확인할래요
융합
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
스마트학생증 재발급
휴학에 대해 알려주세요
학생증 신청
등록금 납부 방법에 대해서 알려주세요
포탈
서울 석식 메뉴
학식 메뉴 확인할래요
대학원 어학시험
영어시험
예술대학 교학지원팀 연락처 알려주세요
공과대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀 연락처
학생증/신분증 발급내역 조회할 수 있나요
학점등록
휴학증명서 발급 안내해주세요
서울 석식 메뉴
증명서 발급 방법 알려주세요
서울 중식 메뉴
오늘 학식
다빈치 석식 메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
이수구분
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
e-Ad 메뉴_e-Advisor 설명
다빈치 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
02-820-5423
중앙대 장학금
경영경제대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀
서울 석식 메뉴
학식 메뉴 확인할래요
장학금 신청내역 조회
내 장학 금액이 궁금해
졸업요건 문의
장학금 확인
서울 석식 메뉴
학식 메뉴 확인할래요
교양대학 교학지원팀 연락처 알려주세요
교내 연락처 검색하고 싶어
서울 석식 메뉴
학식 메뉴 확인할래요
이수 학점 확인하고 싶어요
서울 석식 메뉴
학식 메뉴 확인할래요
건축학부
자유선택 학점 교양인가요?
자유선택
서울캠 내일 중식메뉴
휴학에 대해 알려주세요
서울캠 내일 조식메뉴
서울캠 내일 학식메뉴
이수 학점 확인하고 싶어요
스마트학생증 재발급
학생증 발급
학생지원팀
서울 석식 메뉴
국가근로장학 관련 연락처
서울 중식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
학부에 대해 안내해주세요
식품공학과 학과장님
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
융합전공
서울 석식 메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
문화콘텐츠
학식 메뉴 확인할래요
학생증
종합정보
신입생 오티
단과대학 교학지원팀 전화번호
이수구분
이수구분
서울 석식 메뉴
학식 메뉴 확인할래요
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
도서관 이용 방법 알려주세요
도서 대출 안내
학점등록은 언제, 어떻게 하나요?
도서관 이용 방법 알려주세요
push 확인하고 싶어
서울 석식 메뉴
생활관 홈페이지
학식 메뉴 확인할래요
예술대학 교학지원팀 연락처 알려주세요
구독
학사일정
단과대학 교학지원팀 연락처
우리학교 학사일정을 구독하고싶어
학점등록
포탈
졸업사정
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
도서관 이용 방법 알려주세요
이수 학점 확인하고 싶어요
학점등록
eid 발급
서울 석식 메뉴
학식 메뉴 확인할래요
이수 학점 확인하고 싶어요
자선
서울 석식 메뉴
학식 메뉴 확인할래요
사회복지학부 교수진
증명서 발급 방법 알려주세요
사회복지학부
다빈치 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
서울캠 내일 석식메뉴
학식 메뉴 확인할래요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
필수교양
서울캠퍼스 천원의아침밥 먹을래
천원의 아침밥 대학원생
이수 학점 확인하고 싶어요
핵심교양
서울캠 내일 조식메뉴
서울캠 내일 학식메뉴
서울 조식 메뉴
학식
경영경제대학 안내해주세요
단과대학 홈페이지
융합전공 20
이수 학점 확인하고 싶어요
복사실
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
도서 대출 안내
도서관 이용 안내
다빈치 석식 메뉴
학식 메뉴 확인할래요
서울도서관 운영시간
도서관 운영시간 안내
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
도서 대출 방법 알려줘
내 장학 금액이 궁금해
도서 대출 한도 안내
증명서 발급 방법 알려주세요
도서 대출 안내
MACH교양
김대원
전공 성적 조회
교직원/교수님
교내 연락처 검색하고 싶어
도서관 이용 방법 알려주세요
서울 석식 메뉴
석식
생명공학대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀 연락처
식품영양학과 사무실
서울 석식 메뉴
학식 메뉴 확인할래요
공과대학 교학지원팀 연락처 알려주세요
학생식당 위치
학식당 운영시간
대학별 교학지원팀 연락처 알려주세요
학과에 대해 알려주세요
연구조교
대학원 교학지원팀 연락처
단과대학 교학지원팀 연락처
대학원 교학지원팀 연락처
장학 문의
보건실이 어디야?
이수 학점 확인하고 싶어요
서울 석식 메뉴
학식 메뉴 확인할래요
셔틀버스 시간표와 노선을 알고 싶어요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
서울캠퍼스 학식 메뉴 확인할래요
학생지원팀 위치
봉사
학생식당 위치
다빈치 내일 중식메뉴
다빈치캠 내일 학식메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
졸업 가능여부(졸업사정)를 확인하고 싶어요
경영학부사무실
대여
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
실험실 안전 교육
교양대학 교학지원팀 연락처 알려주세요
서울 석식 메뉴
실험실
교내 연락처 검색하고 싶어
이수 학점 확인하고 싶어요
학식 메뉴 확인할래요
필수로 이수해야하는 교양과목이 무엇인가요?
MACH교양
교양
부서명
디빈치교양
학생증 재발급
교양
e class
우리은행은 어디있나요?
교내 연락처 검색하고 싶어
스마트학생증 재발급
학생증 발급
모바일 학생증/신분증 분실등록 및 기변신청은 어떻게 하나요
학적등록
서울 석식 메뉴
계절학기
학식 메뉴 확인할래요
소프트웨어대학 교학지원팀 연락처 알려주세요
교내 연락처 검색하고 싶어
이수 학점 확인하고 싶어요
교환학생
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
수강신청은 언제, 어떻게 하나요?
계획서
다빈치 석식 메뉴
다빈치 중식 메뉴
학식 메뉴 확인할래요
다빈치 석식 메뉴
다빈치 중식 메뉴
도서(책) 대출 방법 알려주세요
법학도서관 시설안내
도서관 시설 알려줘
법학도서관
다빈치 중식 메뉴
서울캠퍼스 대출자료실
학식 메뉴 확인할래요
도서관 시설 알려줘
법학도서관
융합전공
다빈치 조식 메뉴
문화다양성 전공
융합전공
졸업인정제 영어 서류 제출
강의계획서를 확인하고 싶어요
졸업인증제
전공심화는 무엇인가요?
심화전공
다빈치 석식 메뉴
건축학부
학식 메뉴 확인할래요
성적우수장학금
재학생 장학금 안내
서울 석식 메뉴
분할납부 알아보기
역량강화장학금
학식 메뉴 확인할래요
수강신청 정정
석시
재학생 장학금
수강신청
등록금 안내
학생지원팀 연락처
외부장학 공지
중앙대 장학금
재직자
신청 가능한 장학금이 있나요
성적우수장학금
재학생 장학금
장학 문의
시설관리처 시설팀 연락처
다빈치 내일 조식메뉴
다빈치캠 내일 학식메뉴
히터
학식 메뉴 확인할래요
식품영양학과 교학지원팀 이메일 주소 알려줘
수강신청
졸업하기 위해 필요한 요건이 어떻게 되나요?
서울 석식 메뉴
학식 메뉴 확인할래요
식품영양학과 이메일 알려줘
서울 석식 메뉴
학식 메뉴 확인할래요
도서 대출 한도 안내
이메일 주소
도서관 대출 기간
학생증/신분증 발급내역 조회할 수 있나요
추가학기 등록금
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
법학도서관 운영시간
졸업하기 위해 필요한 요건이 어떻게 되나요
강신청은 언제, 어떻게 하나요?
성적장학금 감면 기준
서울도서관 운영시간
도서관 운영시간 안내
도서관 이용 방법 알려주세요
성적우수장학금
생명공학대학 교학지원팀 연락처 알려주세요
재학생 장학금
등록금 금액(등록금 고지서) 확인하고 싶어요
중앙대 장학금
교학지원팀
장학금 납부
논문
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
성적 장학금
복학을 취소하고 싶어요
졸업 요구
전산팀
e-id 로그인이 안됨
다빈치캠퍼스 내일 석식메뉴
다빈치캠퍼스 내일 학식메뉴
다빈치 중식 메뉴
논문
개인마당
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
수강정정
TOEIC시험 시간
소프트웨어대학 교학지원팀 연락처 알려주세요
예술대학 교학지원팀 연락처 알려주세요
사범대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀 연락처
공과대학 교학지원팀 연락처 알려주세요
학부사무실
TOEIC시험
캠퍼스맵(건물 위치) 보여주세요
학점등록
소프트웨어 학사
정정
학점등록금
일취월장 장학금 기간
학점 당 등록금
공과대학 교학지원팀 연락처 알려주세요
단과대 교학지원팀 전화번호 알려줘
서울 석식 메뉴
학식 메뉴 확인할래요
졸업하기 위해 필요한 요건이 어떻게 되나요?
다빈치캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
예술대학 교학지원팀 연락처 알려주세요
패스트푸드
학생증
졸업 가능여부(졸업사정)를 확인하고 싶어요
교내 편의시설 안내
전화번호 뭐야
e클래스를 사용하고싶어요
교학지원팀 시간
등록금 납부 방법에 대해서 알려주세요
학사팀
부서명
봉사
서울 석식 메뉴
다빈치 내일 조식메뉴
학식 메뉴 확인할래요
다빈치캠 내일 학식메뉴
교내 연락처 검색하고 싶어
학식 메뉴 확인할래요
학사 내역
e-id 로그아웃
e-id가 자동으로 로그아웃돼요
서울 석식 메뉴
학식 메뉴 확인할래요
카우버거 메뉴
등록금 반환
영어능력인증
서울 석식 메뉴
카우버거
서울 석식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
언필제
이수구분변경 신청서
이수 학점 확인하고 싶어요
서울 석식 메뉴
학식 메뉴 확인할래요
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
도서관
계절학기 시간표
졸업요건
부서별 위치 및 전화번호
이메일
캠퍼스맵(건물 위치) 보여주세요
다빈치캠퍼스 내일 석식메뉴
사회복지학부 홈페이지
서울 석식 메뉴
다빈치캠 내일 학식메뉴
학식 메뉴 확인할래요
카우버거 와이파이
기타 장학금 안내
기숙사 금액
미래설계장학금
다빈치캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
기숙사
서울 석식 메뉴
대학원 교학지원팀 알려줘
복지장학금
졸업 학점
재학생장학금
일취월장장학금
재학생 장학금 안내
대여
와이파이 사용방법이 궁금해요
TOEIC
중앙나래장학금
재학생 장학금
글로벌예술학부
장학금 정보가 궁금해요
캠퍼스맵(건물위치) 보여주세요
수강신청
구내서점
교내 편의시설 안내
구내서점
이수 학점 확인하고 싶어요
구내서점
어떤 복지시설(편의시설)이 있나요?
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치캠퍼스 서점
청룡사회봉사
e-Ad 메뉴_e-Advisor 설명
경영학과 이수학점
경영학과
학식 메뉴 확인할래요
Creative complex
등록금 분할납부
타장학금수혜여부확인서
교직원/교수님
교내 연락처 검색하고 싶어
다빈치 석식 메뉴
학식 메뉴 확인할래요
학생증 재발금
서울 석식 메뉴
310관 2층
학식 메뉴 확인할래요
등록금납입증명서
서울 석식 메뉴
학식 메뉴 확인할래요
월별 학사 일정이 궁금해요
성적평가 기준이 어떻게 되나요
졸업학기 등록금
교직원/교수님
서울 중식 메뉴
학식 메뉴 확인할래요
이관
교내 연락처 검색하고 싶어
다빈치 석식 메뉴
학식 메뉴 확인할래요
교환학생 학점인정에 대해 알려주세요
졸업인정제 면제에 대해 알려주세요
서울 석식 메뉴
학점이관
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
서울 석식 메뉴
서울 중식 메뉴
예전 유튜브에 커틀릿 3500원학식은 뭐지
자연과학대학 교학지원팀 연락처 알려주세요
다빈치 석식 메뉴
학식 메뉴 확인할래요
학생식당 위치
다른대학 교학지원팀 연락처
경영경제대학 교학지원팀 연락처 알려주세요
생명공학대학 교학지원팀 연락처 알려주세요
학식 메뉴 확인할래요
단과대학 교학지원팀
졸업하기 위해 필요한 요건이 어떻게 되나요?
졸업 학점
대학별 교학지원팀 연락처 알려주세요
졸업 요건을 충족하지 못했을 때는 어떻게 되나요
CAU 광장
인권센터 안내
휴학 신청을 취소하고 싶어요
기숙사
미수강자
월별 학사 일정이 궁금해요
학생예비군
모바일학생증 발급방법
모바일 학생증/신분증 분실등록 및 기변신청은 어떻게 하나요
e클래스를 사용하고싶어요
학생증 재발급
e-Ad 메뉴_e-Advisor 설명
부서명
졸업논문
e-Ad 메뉴_e-Advisor 설명
서울 석식 메뉴
학식 메뉴 확인할래요
대학규정 및 학칙에 대해 알려주세요
이수 학점 확인하고 싶어요
이수 학점 확인하고 싶어요
등록금납입내역 확인할래요
경영경제대학 교학지원팀 연락처 알려주세요
서울캠 내일 석식메뉴
등록금 금액(등록금 고지서) 확인하고 싶어요
교내 연락처 검색하고 싶어
수강신청
졸업하기 위해 필요한 요건이 어떻게 되나요?
서울캠 내일 중식메뉴
단과대학별 공지사항 있나요?
다빈치 석식 메뉴
서울캠 내일 조식메뉴
학식 메뉴 확인할래요
서울캠 내일 학식메뉴
지식경영학부
서울 석식 메뉴
졸업 논문
서울캠퍼스 천원의아침밥 먹을래
최소 학점
학식 메뉴 확인할래요
수강신청
서울 석식 메뉴
학식 메뉴 확인할래요
단과대학별 공지사항 있나요?
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
최소 학점
학사
다빈치 석식 메뉴
학식 메뉴 확인할래요
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
졸업을 위한 논문, 시험, 작품에 대해서 알려주세요
부서명
도서관 이용 불가
폐강
서울 석식 메뉴
폐강 인원
강의시간표 확인하고 싶어요
소프트웨어학부 졸업 조건
학식 메뉴 확인할래요
서울 석식 메뉴
교내 연락처 검색하고 싶어
학식 메뉴 확인할래요
e클래스를 사용하고싶어요
도서관 음료 반입
도서 대출 한도 안내
봉사
도서 대출 안내
도서관 대출
grs a형 b형
학점변환
봉사
학점등록
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
ㄳ
선수과목
해외협정대학 단기파견 프로그램 지원자격 알려주세요
예술대학 교학지원팀 연락처 알려주세요
사회과학대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀 연락처
서울 석식 메뉴
학식 메뉴 확인할래요
김지현
교직원/교수님
교수 연락처
학점등록
인재개발팀
교직원/교수님
외국인장학금
학생증 재발급
재학생 장학금
교수님의 연락처
신입생 장학금
대학 및 학과(부)별 졸업이수학점 알려주세요
장학금 정보가 궁금해요
등록금 납부 방법에 대해서 알려주세요
전공 평점
서울 석식 메뉴
학식 메뉴 확인할래요
수강신청은 언제, 어떻게 하나요?
강의시간표 확인하고 싶어요
대학원 학사 안내
학식 시간
우편물실
다빈치 석식 메뉴
다빈치캠퍼스 학식메뉴
학식 시간
졸업하기 위해 필요한 요건이 어떻게 되나요?
다빈치 석식 메뉴
공과대학 교학지원팀이 어디있나요
학식 메뉴 확인할래요
iris
분실등록 어디서해?
아이리스
학생증
재수강리스트
등록금 납입확인서 발급 안내해주세요
증명서 발급 방법 알려주세요
서울 석식 메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
도서관 사물함
강의계획서를 확인하고 싶어요
어학연수
어학연수
자퇴
중앙사랑장학금
중앙사랑
졸업사정 신청서
이아이디 기기변경
aacbs
이아이디
편입생 학점인정을 안내해주세요
아이림스
교내 카페
강의계획서를 확인하고 싶어요
이수 학점 확인하고 싶어요
유통관리연계전공 소속대학
e클래스를 사용하고싶어요
편입생 학점인정을 안내해주세요
강의계획서를 확인하고 싶어요
이수 학점 확인하고 싶어요
다빈치 석식 메뉴
학식 메뉴 확인할래요
1학점 인정
와이파이
선수과목
이수 학점 확인하고 싶어요
도서관 카페
중앙사랑장학금
중앙사랑
봉사활동을 학점으로 인정받을 수 있나요
행정인턴
공과대학 교학지원팀 연락처 알려주세요
사회과학대학 교학지원팀
사회과학대학
교내 연락처 검색하고 싶어
행정인턴
행정인턴 예비군
행정인턴
강의계획서
서울 석식 메뉴
학식 메뉴 확인할래요
인문대학 교학지원팀
규정
예비군
중앙사랑장학금
중앙사랑
교직원 예비군
교직원 예비군
다빈치 석식 메뉴
학식 메뉴 확인할래요
전공심화란 무엇인가요
중앙사랑장학금
중앙사랑
졸업하기 위해 필요한 요건이 어떻게 되나요?
졸업이수
서울 석식 메뉴
학식 메뉴 확인할래요
삼수강신청 방법
삼수강
중앙사랑장학금
중앙사랑
예비군 전입신고
한자
이수 학점 확인하고 싶어요
한자 자격증 대체 과목
법학전문대학원
학점등록은 언제/어떻게 하나요
강신청은 언제, 어떻게 하나요?
법학전문
등록금 납부 방법에 대해서 알려주세요
기타납입금 알아보기
법
등록금
법학관
신입생 장학금
법과대학
강의계획서를 확인하고 싶어요
장학금 정보가 궁금해요
국가시험
약학대학 및 R&D센터 방호실
이수 학점 확인하고 싶어요
102관 205호
102관 205호 알려줘
캠퍼스맵(건물 위치) 보여주세요
이승억 교수
교직원/교수님
카우버거
이승억 교수님 메일
학생식당 위치
이승억
연장사유 확인서
강의시간표 확인하고 싶어요
외국인 비자 연장 사유
e클래스를 사용하고싶어요
일정 연장 확인서
이승억 교수
이승억 교수
서울 석식 메뉴
학식 메뉴 확인할래요
국제처
추가납부
국제처
서울캠 지도
수강신청은 언제, 어떻게 하나요?
무인발급기 위치
경영경제대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀 연락처
단과대학 교학지원팀 연락처
이수 학점 확인하고 싶어요
법학
금융 ai
학점등록은 언제/어떻게 하나요
학점조회
연구실안전 법정교육이 뭔가요
연구실법정
법학이수
복학생 스마트학생증
스마트학생증 신규발급
서울 중식 메뉴
학생증 교부는 언제되나요?
서울 석식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
일반휴학에서 군휴학으로 변경하고 싶어
무인보관
e상담센터 상담 신청할래요
문의
학점등록은 언제, 어떻게 하나요?
경영경제대학 교학지원팀 연락처 알려주세요
대학별 교학지원팀 연락처 알려주세요
강의계획서
조기졸업 기준
추가등록
등록금 납부 방법에 대해서 알려주세요
6학점 이하일 때 등록금 반환 기준이 궁금해
등록금 납부 방법에 대해서 알려주세요
학점등록
추가등록
월별 학사 일정이 궁금해요
졸업예정증명서가 확인이 안 돼
연구등록 추가등록
중앙사랑장학금
재학생장학금
추가등록
서울 중식 메뉴
수강신청은 언제, 어떻게 하나요?
중앙사랑장학금
학식 메뉴 확인할래요
중앙사랑
연구등록
스마트학생증/신분증 신규발급 신청방법 알려주세요
재학생 추가등록
추가등록
서울 석식 메뉴
학식 메뉴 확인할래요
학생증이 뭔가요
모바일 학생증/신분증이 뭔가요
강의계획허
수상 이력
성적증명서 발급할래요
부전공 융합전골
역량강화장학금
미크로스칼리지
미크로스칼리지
포털 어플
전공평점
재학생 장학금 안내
역량강화
학사팀
졸업
건축공학부
우등졸업
경영학과 교학지원팀
경영 교학지원팀
최우등 졸업
커뮤니케이션 인 잉글리쉬를 토익스피킹으로 대체하고 싶은데 몇점 이상 받아야해?
등록금 납부 방법에 대해서 알려주세요
졸업하기 위해 필요한 요건이 어떻게 되나요?
수강신청은 언제, 어떻게 하나요?
서울 석식 메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
복사실
서울캠 내일 석식메뉴
학식 메뉴 확인할래요
편의시설 안내
서울캠 내일 학식메뉴
207관 프린트
계획서
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식
복수전공
서울 석식 메뉴
학식 메뉴 확인할래요
학사팀
출석
서울캠퍼스 스터디룸 예약
스터디룸/팀플룸 예약 안내
역량강화
서울 석식 메뉴
서울 석식 메뉴
커뮤니케이견잉잉글리시 과목 인정제
학식 메뉴 확인할래요
학식 메뉴 확인할래요
장학금 정보가 궁금해요
학비할부
삼수강
서울캠퍼스 주차장 이용 안내
주차 정기권 신청
다빈치 강의
중앙대학교 다빈치
니진스키
학자금대출
졸업 관련 증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
201관
서울캠퍼스 주차장 이용 안내
부서명
주차 정기권 금액
부서별 위치 및 전화번호
수강정정
201관
q&a게시판
성적증명서
201관 전화번호 알려줘
서울 석식 메뉴
기타 부득이한 사유로 총장의 허가
학식 메뉴 확인할래요
201관 전화번호
다전공(복수/연계/융합/자기설계/부전공)이 뭔가요?
10번 항목 관련 서류
졸업 예정증명서
병결
연계 전공
예술대학 교학지원팀 연락처 알려주세요
한자
대학별 교학지원팀 연락처 알려주세요
과동아리
전과 문의는 어디서 하나요
산업협력인턴
공공인재학부
부서별 연락처안내
서울 석식 메뉴
학식 메뉴 확인할래요
동아리개설
계절학기 수강신청은 언제, 어떻게 하나요?
복수전공
전체 시간표
학사팀 전화번호
성적증명서
P학점
동아리
전과는 어떻게 할 수 있나요
P/f
이수 학점 확인하고 싶어요
강의시간표 확인하고 싶어요
이수구분
전과전의 과
서울중앙대부터 안산까지 학교왓다갓다할려면 편리한교통수단이 잇을까요?
전적과
셔틀
서울 석식 메뉴
학식 메뉴 확인할래요
안녕하세요? 혹시 서울중앙대에서부터 안산까지 학교왓다
봉사 학점
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
예비군연대
부전공 이수 학점
소프트웨어학과 복수전공 조건
부전공 학점
예비군
교내 연락처 검색하고 싶어
사회개발시스템공학부 사무실 위치
전과
스마트학생증 신규발급 신청방법 알려주세요
복수전공
복수전동
학적사항
학생증/신분증 선택
서울 석식 메뉴
융합전공 조회
학식 메뉴 확인할래요
내 융합전공
출결확인
강신청은 언제, 어떻게 하나요?
교양대학 교학지원팀 연락처 알려주세요
서울캠퍼스 주차장 이용 안내
주차
졸업요건
교내 연락처 검색하고 싶어
졸업인정학점
소프트웨어대학 교학지원팀 연락처 알려주세요
실험과목
대학별 교학지원팀 연락처 알려주세요
기초 컴퓨터 프로그래밍 전공
실기과목
전공학점
학생증 발급
교환학생 및 방문학생에 대해 알려주세요
실기과목 성적
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
개인장학
개인자싹
이수 학점 확인하고 싶어요
이수 학점 확인하고 싶어요
단과대학별 시험일정 어떻게 확인하나요?
서울 석식 메뉴
학식 메뉴 확인할래요
부서명
절대평가
교내 연락처 검색하고 싶어
이수 학점 확인하고 싶어요
수강한 과목의 이수 구분을 변경하고 싶어요
학식 메뉴 확인할래요
서울 석식 메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
교환학생 및 방문학생에 대해 알려주세요
국제학생증
융합 전공
e클래스를 사용하고싶어요
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
다빈치 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
출석인정제
인문대학 교학지원팀 연락처 알려주세요
서울 조식 메뉴
대학별 교학지원팀 연락처 알려주세요
학식 메뉴 확인할래요
과사무실
교내 연락처 검색하고 싶어
입학 전형별 경쟁률이 궁금해요
내신
강의평가
서울 석식 메뉴
학식 메뉴 확인할래요
복수전공 포기
대학원 예비군 편입
재학증명서
분실물
재학증명서
학생증 교부는 언제되나요?
학생증 신청
인적성
성인지
인문대학 교학지원팀 연락처 알려주세요
신청 가능한 장학금이 있나요
단과대학 교학지원팀 전화번호
학생증 재발급
이수 학점 확인하고 싶어요
수강한 과목의 이수 구분을 변경하고 싶어요
기기변경
서울캠 내일 석식메뉴
서울캠 내일 학식메뉴
등록금납입내역 확인할래요
등록금 감면
이수 학점 확인하고 싶어요
추가학기
서울 석식 메뉴
졸업 직전 학기 등록금
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
흡연
도서관 이용 방법 알려주세요
학식 메뉴 확인할래요
예술대학 교학지원팀 연락처 알려주세요
졸업인정제(영어/한자)에 대해서 알려주세요
학과사무실
조기졸업
중앙사랑 장학금
경제학부
경제학부
재학생 장학금
중앙대 장학금
흡연
연구실 안전교육
중앙사랑장학금
국가시험장학금
근로장학금
미래설계장학금
중앙나래장학금
재학생장학금
전자신문/또는 전자책(e-book)은 어디서 볼 수 있나요
성적평가 기준
이향숙
교직원/교수님
중앙사랑장학금
교내 연락처 검색하고 싶어
중앙사랑
성적
우리은행 위치
학생증/신분증 사진 변경 어떻게 하나요
단과대학별 공지사항 있나요?
서울 석식 메뉴
학식 메뉴 확인할래요
강의계획서를 확인하고 싶어요
전공시험
전자교탁
서울 석식 메뉴
학식 메뉴 확인할래요
강의계획서
서울 석식 메뉴
학식 메뉴 확인할래요
야야 내 연계 전공 합격한 거 뭐 있냐고
졸업인정제
모바일서비스
컴퓨터실습실에 컴퓨터가 몇대야?
연계전공 신청
강의 정원이 과거에 어땠는지 알고 싶어
나 연계전공 붙은 거 뭐있냐
생명공학대학 교학지원팀 연락처 알려주세요
전자출석 어떻게 하나요
교학지원팀
스마트학생증 신규발급
학생증
서울 석식 메뉴
학식 메뉴 확인할래요
강의교재 알려주세요
전자출삭
경제학부 학생회실
경영학부 전화번호좀 알려줘
다전공 합격 확인
융전 합격
다빈치 석식 메뉴
학식 메뉴 확인할래요
종합정보시스템
학식
재학증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
경영학부
졸업인정제 면제에 대해 알려주세요
경영경제대학 교학지원팀
서울 석식 메뉴
경영경제대학
다전공
서울 중식 메뉴
다빈치학술정보원
학식 메뉴 확인할래요
이용Tip
졸업인정제
경영학부
와이파이 사용방법이 궁금해요
와이파이
e클래스를 사용하고싶어요
학점
재학등면서
실험실 안전교육
인권센터 안내
인권센터 안내
전과는 어떻게 할 수 있나요
도서관 위치
push 확인하고 싶어
강의계획서를 확인하고 싶어요
서울 석식 메뉴
종합정보시스템
실험실 안전
학점등록은 언제/어떻게 하나요
예비군
다빈치캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
서울캠퍼스 학식메뉴
식당 위치
그래 안녕
너 진짜 착하다
수고해
민원
고마워
캠퍼스맵(건물위치) 보여주세요
g힘들어
나의 도서관 이용현황이 궁금해요
중앙대 영어영문
무인도서예약
셔틀버스
다빈치 중식 메뉴
다빈치 석식 메뉴
학식 메뉴 확인할래요
단과대학 교학지원팀
성적우수장학금
재학생 장학금
학생지원팀 연락처
다빈치캠퍼스 석식 메뉴
장학금 정보가 궁금해요
국제대학원
다빈치 중식 메뉴
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
단과대학 교학지원팀 연락처 알려주세요
서울 중식 메뉴
학식 메뉴 확인할래요
예술대학 교학지원팀 연락처 알려주세요
근로
이수체계도 확인하기
생리대
필수로 이수해야하는 교양과목이 무엇인가요?
전공필수 전공기초 전공
이수구분 변경신청서
찰리가 연락처 정보를 안내해줄게요
이수구분변경신청
강의계획서
02-820-0282
부서별 연락처안내
다빈치 석식 메뉴
건강센터
건강세넡
학식 메뉴 확인할래요
경영경제대학 교학지원팀 연락처 알려주세요
교학지원팀
대학원 장학금
gpt
인문대학 교학지원팀 연락처 알려주세요
서울캠 내일 중식메뉴
근로
정치적중립
서울캠 내일 학식메뉴
대학별 교학지원팀 연락처 알려주세요
일취월장
이수 학점 확인하고 싶어요
서울 석식 메뉴
학식
윤석열
이수구분
박성춘
카드 재발급
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
백준
일본문화와예술의이해
서울 생활관에 대한 정보는 어디서 보나요
다빈치 생활관에 대한 정보는 어디서 보나요
다빈치
mro
대학교
학교
중앙대
박성춘
도서관
이수 학점 확인하고 싶어요
통일공대
소프트웨어대학
공과대학
학점
ai학부
이수 학점 확인하고 싶어요
강의교재 알려주세요
소프트웨어학부
이수 학점 확인하고 싶어요
소프트웨어
한자증명서 제출
인공지능지원팀
다빈치캠퍼스 내일 석식메뉴
다빈치 내일 중식메뉴
수강신청에 대해서 모르는 부분이 있ㄴ
다빈치 내일 조식메뉴
다빈치 내일 학식메뉴
시설관리팀
부서명
031-670-7425
문헌정보학과
학사지원팀 전화번호
기숙사
원격수업 로그인 방법
학생증
공동인증서 로그인
산업보안학과
수강신청에 대해서 물어보고싶은게 있다
산업보안학과 사무실
서울 석식 메뉴
학식 메뉴 확인할래요
산업보안학과 사무실
휴학취소
이수 학점 확인하고 싶어요
기숙사
성적증명서 발급할래요
스마트학생증 신규발급 신청방법 알려주세요
교직원 신분증
모바일학생증 다운로드 방법
모바일 학생증/신분증이 뭔가요
총무팀 이메일
와이파이 사용방법이 궁금해요
서울 중식 메뉴
총무팀
부서별 연락처안내
부서명
총무처
교내 연락처 검색하고 싶어
성적 등급
박기석
다빈치 중식 메뉴
인문 융합전공
경영학과 사무실
다빈치 중식 메뉴
단과대학 교학지원팀 연락처 알려주세요
경영학부
부서명
교내 연락처 검색하고 싶어
다빈치 석식 메뉴
다빈치 석식 메뉴
다빈치 중식 메뉴
학식 메뉴 확인할래요
문헌정보학과
과사무실
필수로 이수해야하는 교양과목이 무엇인가요?
소프트웨어 융합전공
다빈치 중식 메뉴
학식 메뉴 확인할래요
과목의 수업 현황을 확인하고 싶어요
과목 공지는 어디서 봐?
서울캠퍼스 참고자료실
연계전공
서울캠퍼스 대출자료실
도서관 시설 안내
학생증 발급
유통관리연전
서울도서관 운영시간
도서관 운영시간 안내
도서관 이용 방법 알려주세요
중앙도서관 프린트
교양대학
국제물류
모바일 학생증/신분증 분실등록 및 기변신청은 어떻게 하나요
인문대학 교학지원팀 연락처 알려주세요
대학별 교학지원팀 연락처 알려주세요
310관 519호
인문대학 교학지원팀 연락처 알려주세요
외국인장학금
단과대학 교학지원팀 전화번호
재학생장학금
시간표
단과대학별 공지사항 있나요?
사회과학 교학팀
졸업을 위한 논문/시험/작품에 대해서 알려주세요
사회과학 교학팀 내선 번호
외국인장학금
학생증 재발급
졸업을 위한 논문, 시험, 작품에 대해서 알려주세요
대학원 교학지원팀 연락처
실험복
전화번호 뭐야
박화연
교양대학 행정실 이메일
교직원/교수님
교내 연락처 검색하고 싶어
강의계획서를 확인하고 싶어요
졸업예정
이수구분 변경 신청
소프트웨어대학 학과사무실
박화연
소프트웨어대학 교학지원팀 연락처 알려주세요
교직원/교수님
교내 연락처 검색하고 싶어
소프트 학과사무실
정정기간
Wi-Fi사용방법
융합전공
유통관리연계전공
스마트학생증 재발급
공과대학
wifi사용방법
분실등록 어디서해?
일반대학원교육지원팀
학생증
중앙대학교 공과대학
학생증/신분증 사진 변경 방법 알아보기
수강신청
포탈 사진 변경
학생증/신분증 사진 변경 방법 알아보기
출석부 사진 변경
교양대학 교학지원팀 연락처 알려주세요
단과대학
졸업 요건 조회
졸업 학점
서울 석식 메뉴
학식 메뉴 확인할래요
교양대학 교학지원팀
교양대학 안내해주세요
학부 단과대학에 대해 알려주세요
근로
수강신청 교양
공통교양
교양학부
다빈치 교양학부
휴일
스마트학생증 재발급
학생증 발급
인문대학 교학지원팀 연락처 알려주세요
등록금 납부 방법에 대해서 알려주세요
중앙대학교 이념 및 역사를 알고싶어요
학교 개관
성적증명서
기념일
학과 사무실
scifinder
단과대학 교학지원팀 전화번호
이수구분 신청
성적증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
대학원 장학금
성인지
봉사장학금
재학생장학금
봉사장학금
재학생장학금
졸업예정증명서
학점등록
캠퍼스 지도
봉사장학금
위치
단과대학 교학지원팀 전화번호
내가 신청한 융합전공
재학생 장학금
융합전공 합격내역
이수구분 신청서
학생회 장학금
융합전공
강의시간표 확인하고 싶어요
융합전공
등록금
월별 학사 일정이 궁금해요
정정
졸업하기 위해 필요한 요건이 어떻게 되나요?
졸업유예
교내 셔틀
중앙사랑장학금
중앙사랑
서울 조식
교시
숙소비
학생번호
학번
숙소
재학증명서 영어
학번
사범대학 교직팀
교직
융합전공 신청 내역
구내서점
교직팀
재학증명서 발급 안내해주세요
서울 중식 메뉴
증명서 발급 방법 알려주세요
학사
학식 메뉴 확인할래요
강의교재 알려주세요
법과대학 교학지원팀 연락처 알려주세요
교직팀
교내 연락처 검색하고 싶어
졸업관련
월별 학사 일정이 궁금해요
교재 구매
다전공(복수/연계/융합/자기설계/부전공) 신청할래요
이수 학점 확인하고 싶어요
학생식당 위치
식당 운영시간
사회과학대학 교학지원팀 연락처 알려주세요
변리사
교학지원팀
Atm
학생증 발급
이수과목
대체과목
과목번호
복수전공이 무엇인가요?
과목번호 조회
전공 안내
이수구분변경신청서 방법
모바일학생증 다운로드 방법
모바일 학생증
복수전공 신청
서울캠퍼스 주차장 이용 안내
주차
e클래스를 사용하고싶어요
box
it
학부에 대해 안내해주세요
예술대학 교학지원팀 연락처 알려주세요
학과
단과대학별 공지사항 있나요?
단과대학 교학지원팀 연락처
수강 문의
졸업논문
azure id
문의
강의교재 알려주세요
시험 휴학
성적증명서 발급 안내해주세요
학점등록신청서
증명서 발급 방법 알려주세요
서울 석식 메뉴
학식 메뉴 확인할래요
조교증
휴학
계좌등록
등록금 납부 방법에 대해서 알려주세요
학생증
조교 학생증
수강신청 문의
정치국제학과 사무실
학생 문의
시간표 문의
성적증명서
예비군
융합전공
학교 상점
융합전공
학식 메뉴 확인하는 곳
질병휴학
서울 중식 메뉴
용우당 위치
학식 메뉴 확인할래요
학부에 대해 안내해주세요
서울캠 내일 학식메뉴
서울캠퍼스 천원의아침밥 먹을래
학과
학식 메뉴 확인할래요
나의 지도교수님은 누구인가요
사범대학 안내해주세요
단과대학 홈페이지
부전공
부전공
부전공이 무엇인가요?
전공 안내
부전공 신청
월별 학사 일정이 궁금해요
인권센터 안내
부전공
강의 계호기서
강의계획서
학생지원팀
102관
인문대학교 교학지원팀
교내 근로
글로벌예술 메일
사회과학대학 교학지원팀 연락처 알려주세요
글로벌
단과대학 교학지원팀 연락처
예술대학 교학지원팀 연락처 알려주세요
모바일 학생증
교내 연락처 검색하고 싶어
연계전공
졸업학점을 보고싶어
학식 운영시간
학점등록은 언제/어떻게 하나요
스마트학생증 재발급
스마트학생증 신규발급
학생증
선납입금 정정기간
인턴
청강
등록금납입내역 확인할래요
등록금 금액 확인하고 싶어요
청강신청
등록금 납부 방법에 대해서 알려주세요
e클래스를 사용하고싶어요
성적증명서
연구실안전 법정교육이 뭔가요
성적증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
참여연구원을 위한 연구윤리
서울 조식 메뉴
어제 조식은?
수강정정하는 방법
서울 석식 메뉴
학식 메뉴 확인할래요
학생증 교부는 언제되나요?
서울 조식 메뉴
재학증명서 발급 안내해주세요
학식 메뉴 확인할래요
증명서 발급 방법 알려주세요
아이림스
절대평가
재학증명서 발급 안내해주세요
교양대학 교학지원팀
증명서 발급 방법 알려주세요
학생증 신청
교양대학
재학증명서 발급 안내해주세요
재학증명서 휴학
증명서 발급 방법 알려주세요
추가등록
수강신청
학점등록
이수변경
재학증명서
학생증/신분증 사진 변경 방법 알아보기
재학증명서
포탈 사진 변경 어떻게 하나요
김재휘
교직원/교수님
교내 연락처 검색하고 싶어
김재휘
학식 메뉴 확인할래요
졸업하기 위해 필요한 요건이 어떻게 되나요?
학생지원팀 연락처
장학팀
공공인재학부 졸업사정 기준표
출석
사회과학대학 교학지원팀 연락처 알려주세요
공공인재학부 졸업사정 기준표
출석 확인
경영경제대학 교학지원팀 연락처 알려주세요
포탈 아이디/비밀번호를 잊어버렸어요
창의와 소통알아?
사회과학대학 교학지원팀 연락처 알려주세요
기타납입
대학별 교학지원팀 연락처 알려주세요
졸업요건
인문대학 교학지원팀 연락처 알려주세요
부서명
교내 연락처 검색하고 싶어
인턴쉽
등록
기기변경
다빈치 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식
이수 학점 확인하고 싶어요
공과대학 교학지원팀 연락처 알려주세요
독서 프로그램에 대해 알려주세요
대학별 교학지원팀 연락처 알려주세요
시설팀
전과 이수 과목
회의실
캠퍼스맵(건물 위치) 보여주세요
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
학점인정신청
도서관 이용 방법 알려주세요
봉사활동을 학점으로 인정받을 수 있나요
경영경제대학 교학지원팀 연락처 알려주세요
대학별 교학지원팀 연락처 알려주세요
이용Tip
중앙대 출석인정 관하여 전화해볼 곳
강의실 대관
대학규정 및 학칙에 대해 알려주세요
서점 어디예요?
조교신분증신청
수강신청 정정
졸업을 위한 논문, 시험, 작품에 대해서 알려주세요
졸업하기 위해 필요한 요건이 어떻게 되나요?
이수 학점 확인하고 싶어요
동문회비
인턴쉽
회의실 대여 하고 싶어요
자연과학대학 교학지원팀 연락처 알려주세요
물리학과 과사 전화번호
절대평가
건축학교육 인증
인문대학 교학지원팀 연락처 알려주세요
건축학교육 인증
ROTC
서울 중식 메뉴
학식 메뉴 확인할래요
학생지원팀 연락처
국어국문학과 교수
서울 중식 메뉴
학식 메뉴 확인할래요
단과대학 교학지원팀 전화번호
장학금 정보가 궁금해요
졸업예정증명서 발급 방법
이수변경신청
동창회비
서울 중식 메뉴
학식 메뉴 확인할래요
혜택
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
다빈치캠 내일 학식메뉴
사회과학대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀 연락처
다빈치 석식 메뉴
병원
학식 메뉴 확인할래요
사회복지학부 사무실
성적평가 기준이 어떻게 되나요
등록금납입내역 확인할래요
이수구분신청서
등록금 고지서 옜날
이수구분
조기졸업
학점 등록
교수별 과목
과목 검색
등록금 납부 확인서
장학금 정보가 궁금해요
서울캠퍼스 주차장 이용 안내
주차요금
등록금 감면
성적유형
등록금 감명
학생증발급
졸업예정증명서
재학생 장학금
교원증 발급
감면
중앙대학교 모바일 앱을 알고 싶어요
열람실 좌석배정
도서관 이용 방법 알려주세요
학비
증명서 발급방법과 수수료에 대해 알려주세요
교원증발급
글로벌 경영인증센터
학식 메뉴 확인할래요
학점등록
다른대학 교학지원팀 연락처
경영경제대학 교학지원팀 연락처 알려주세요
졸업학점을 확인하고 시펑
대학별 교학지원팀 연락처 알려주세요
유혜경
출석 불가
개인시간표
학생식당 위치
공공인재
등록금 납입확인서 발급 안내해주세요
추가등록
교육비납입증명서 발급 안내해주세요
공공인재
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학비확인
수업조회
학식 메뉴 확인할래요
수강정정
기기변경
수강신청
한자
경영경제대학 교학지원팀
경영경제대학 안내해주세요
단과대학 홈페이지
융합전공취소신청서
학생식당 위치
나의 수업(e클래스) 강의자료를 확인하고싶어요
소프트웨어 다운로드는 어디에서 받나요
출석 안됨
이수구분
Eclass
나의 수업(e클래스) 과제를 확인하고싶어요
e-class
교직원/교수님
교수 이메일
교내 연락처 검색하고 싶어
예술대학
이메일
서울 석식 메뉴
학식 메뉴 확인할래요
이승우
모바일 서비스
은행 어디있어?
교직원/교수님
교내 연락처 검색하고 싶어
제과점 어디있어?
서울캠퍼스 주차장 이용 안내
졸업예정증명서
주차권
음식점 어디있어?
서울 중식 메뉴
학식 메뉴 확인할래요
e클래스를 사용하고싶어요
학생식당 위치
봉사
패스트푸드
e-adviser
편의점
수강내역
커피점
졸업요건충족
어떤 복지시설(편의시설)이 있나요?
빈 강의실 사용
자연과학대학 교학지원팀 연락처 알려주세요
대학별 교학지원팀 연락처 알려주세요
빈 강의실
학생지원팀 연락처
위정현
교직원/교수님
교내 연락처 검색하고 싶어
장학금문의
다빈치 석식 메뉴
교직원/교수님
사회과학대학 공공인재학부
학식 메뉴 확인할래요
다빈치가상대학
가족장학금
부서명
학식 메뉴 확인할래요
교내 연락처 검색하고 싶어
예술대학 교학지원팀 연락처 알려주세요
장학금정보가 궁금해요
단과대학 교학지원팀 연락처 알려주세요
단과대학별 공지사항 있나요?
공지사항
필수로 이수해야하는 교양과목이 무엇인가요?
단과대학 홈페이지
학부 공지사항
융합전공
등록금 납부 방법에 대해서 알려주세요
졸업인정제(영어/한자)에 대해서 알려주세요
광고홍보학과 사무실
교내 연락처 검색하고 싶어
수강내역 조회
사회과학대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀 연락처
핵심교양 알려줘
학식 메뉴 확인할래요
휴학
졸업하기 위해 필요한 요건이 어떻게 되나요?
학점등록
출석
인문학40권읽기
출석
출석
심리학과
교내 연락처 검색하고 싶어
모바일학생증 안내해주세요
학생증
공공인재학부
단과대학 홈페이지
중간고사
융합전공
경영경제대학 교학지원팀 연락처 알려주세요
6225
학과사무실
수강정정
재학증명서
전공 필수
졸업하기 위해 필요한 요건이 어떻게 되나요?
졸업 가능여부(졸업사정)를 확인하고 싶어요
610관
학생증/신분증 발급내역 조회할 수 있나요
다빈치학술정보원
다빈치 석식 메뉴
학식 메뉴 확인할래요
이수 학점 확인하고 싶어요
대학 및 학과(부)별 졸업이수학점 알려주세요
24년2월에졸업했는데 학위기를못받아서ㅠ 어디서받으면될까요?
이수 학점 확인하고 싶어요
내 전공 필수가 뭐야?
주차관리센터
국제처
주차
스마트학생증 재발급
학생증 등록
졸업하기 위해 필요한 요건이 어떻게 되나요
배차담당
학점조회
이수 학점 확인하고 싶어요
학점등록신청서
학부 학점등록
전공심화
학점등록
서울캠 내일 중식메뉴
금융 기능이 없는 학생증은 어떻게 발급해?
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
학생증 발급 어디서해?
찰리야
국제처 홈 페이지
소프트웨어대학 교학지원팀 연락처 알려주세요
교내 연락처 검색하고 싶어
절대평가
교학지원팀이 뭐야
정원 20명
학샌증발급
교학지원팀
교내 연락처 검색하고 싶어
교내 연락처 검색하고 싶어
등록금 납부
학점인정신청
학점인정
학점등록
서울 중식 메뉴
학식 메뉴 확인할래요
취업계가 있나요
다빈치 석식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학생증/신분증 선택
졸업 성적
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
소프트웨어대학 교학지원팀 연락처 알려주세요
교내 연락처 검색하고 싶어
학과사무실
대학별 교학지원팀 연락처 알려주세요
스마트학생증 신규발급
신입생 학생증
자자 자타
학생증
장학금 정보가 궁금해요
학생증 사진등록
서울 석식 메뉴
학식 메뉴 확인할래요
학생증
학생증 발급
성적증명서
서울 중식 메뉴
학식 메뉴 확인할래요
부서별 위치 및 전화번호
학사팀
310관 안내
다빈치 석식 메뉴
모바일학생증 다운로드 방법
학생증 등록
모바일학생증
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
기숙사
학생식당 위치
용우당
출석 어떻게 하나요
소프트웨어대학 교학지원팀 연락처 알려주세요
교내 연락처 검색하고 싶어
편의시설 안내
서울 중식 메뉴
학식 메뉴 확인할래요
용우당
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치 중식 메뉴
캠퍼스맵(건물위치) 보여주세요
다빈치캠퍼스 학식 메뉴 확인할래요
서울 중식 메뉴
와이파이 사용방법이 궁금해요
학식 메뉴 확인할래요
예비군 전입신고 방법은 무엇인가요
인권센터에 대해 안내해주세요
이수 학점 확인하고 싶어요
서울 중식 메뉴
학식 메뉴 확인할래요
등록금 납부
교과코드
서울 중식 메뉴
학식 메뉴 확인할래요
재학증명서
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
예비군
재학증명서
서울 중식 메뉴
연계전공
학식 메뉴 확인할래요
공공관리
it상담
wifi 사용방법
소프트웨어대학 교학지원팀 연락처 알려주세요
전화번호 뭐야
교내 행정부서의 근무시간을 알려주세요
강의계획서를 확인하고 싶어요
행정실
재학증명서
부서명
성적표
mro
온라인(재택) 강의 수강 방법 알려주세요
소프트웨어대학 교학지원팀 연락처 알려주세요
7201호 어디야
부서명
융합전공
교내 연락처 검색하고 싶어
재택강의
부서별 위치 및 전화번호
국제처
캠퍼스맵(건물 위치) 보여주세요
교직정보는 어디에서 확인하나요?
보관성적 조회
졸업하기 위해 필요한 요건이 어떻게 되나요?
다른대학 교학지원팀 연락처
성적표
교양대학 교학지원팀 연락처 알려주세요
현장실습
mro
서울 석식 메뉴
학식 메뉴 확인할래요
교내 연락처 검색하고 싶어
e-출석부가 뭔가요?
교내 연락처 검색하고 싶어
국가 장학금
출석 어떻게 하나요
서울캠퍼스 주차장 이용 안내
장학금문의
주차 정기권
서울 중식 메뉴
학식 메뉴 확인할래요
장학금
부서명
등록금 환불
도서관 이용 방법 알려주세요
현장실습
성적증명서 발급 안내해주세요
월별 학사 일정이 궁금해요
다빈치러닝
음악치료학개론 끝나는 시간 알려줘
등록금납입내역 확인할래요
등록금 금액(등록금 고지서) 확인하고 싶어요
학적부 인터넷발급
음악치료학개론은 몇시에 끝나?
편입생
학식 메뉴 확인할래요
버스정류장 위치
증명서 발급 방법 알려주세요
서울 중식 메뉴
학식 메뉴 확인할래요
도서관 이용 방법 알려주세요
다빈치 캠퍼스 본관
성적증명서
이수 학점 확인하고 싶어요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
서울도서관 운영시간
도서관 운영시간 안내
편입생 학점인정을 안내해주세요
도서관
등록 기가
산학협력인턴이 뭔가요?
편입
수강취소
수강신청 취수
공동인증서 발급방법
문화콘텐츠 융합전공
휴학신청
산학협력인턴에 대해 알려주세요
e클래스를 사용하고싶어요
경영사무실
재적증명서 발급 안내해주세요
공공인재
증명서 발급 방법 알려주세요
이수 학점 확인하고 싶어요
서울 석식 메뉴
학점
학식 메뉴 확인할래요
증명서 발급 방법 알려주세요
취업계가 있나요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식
이수 학점 확인하고 싶어요
수납 계좌
다빈치캠퍼스 내일 석식메뉴
다빈치 내일 조식메뉴
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
다빈치캠퍼스 내일 학식메뉴
전자자료
편입생인데 학점인정 신청 언제해?
교직원 버스
재학증명서
도서관 위치
군입대 휴학 및 복학은 어떻게 신청하나요
다빈치캠퍼스 천원의아침밥 먹을래
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
도서관 이용 방법 알려주세요
서울 중식 메뉴
재학증명서
학식 메뉴 확인할래요
학식 메뉴 확인할래요
역량강화
등록금납입내역 확인할래요
편입생 학점인정을 안내해주세요
서울캠 내일 석식메뉴
등록금고지서
월별 학사 일정이 궁금해요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
대학 등록금 수납 계죄
사회과학대학 교학지원팀 연락처 알려주세요
생명공학대학 교학지원팀 연락처 알려주세요
어떤 경우에 출석인정이 가능한가요
대학별 교학지원팀 연락처 알려주세요
서울 석식 메뉴
학식 메뉴 확인할래요
교내 연락처 검색하고 싶어
편입생 학점인정을 안내해주세요
졸업하기 위해 필요한 요건이 어떻게 되나요?
도서관 이용 방법 알려주세요
와이파이 사용방법이 궁금해요
신용카드 납부방법 알아보기
서울 중식 메뉴
일취월장
학식 메뉴 확인할래요
등록금 납부 방법에 대해서 알려주세요
한 학기에 최대 몇학점
최대 학점 몇점까지 수강 가능한가요
经济系办公室邮箱
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
e advisior
실감미디어
서울 중식 메뉴
학식 메뉴 확인할래요
전자자료검색
유아교육과 이메일
대학규정 및 학칙에 대해 알려주세요
도서관 이용 방법 알려주세요
유아교육과
복수전공 취소
학과 메일
학생증 재발급
복전 취소 신청서
서울 중식 메뉴
학식
일취월장장학금
재학생 장학금
도서관 이용 방법 알려주세요
학생회 장학금 기준
다빈치캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
서울 중식 메뉴
계절 학기 최대 학점
학식 메뉴 확인할래요
도서관 이용 방법 알려주세요
졸업하기 위해 필요한 요건이 어떻게 되나요?
지도
이수변경 신청서
한국사
학사관이 어디야
졸업신청
학생증 재발급 수령
심리학과 졸업
서울 석식 메뉴
총무부 위치
강의계획서를 확인하고 싶어요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
국가장학금
등록금납입내역 확인할래요
ca 인증서
등록금 금액(등록금 고지서) 확인하고 싶어요
은행
교내 연락처 검색하고 싶어
다빈치 중식 메뉴
학식 메뉴 확인할래요
원격미래교육원 수업은 맥북으로 듣지 못하나요?
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
e클래스를 사용하고싶어요
성적증명서 전자
서울 석식 메뉴
서울 중식 메뉴
학식
심리상담은 어디서 받을 수 있나요
성적증명서 발급할래요
학생식당
서울캠 내일 중식메뉴
스마트학생증 신규발급
서울캠 내일 학식메뉴
스마트학생증 재발급
성적
서울 중식 메뉴
학식 메뉴 확인할래요
학생증
강신청은 언제, 어떻게 하나요?
절대평가
수강신청
서울캠 내일 중식메뉴
서울캠 내일 석식메뉴
졸업예정증명서
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
서울캠 내일 석식메뉴
서울캠 내일 학식메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
wifi 사용방법
서울 중식 메뉴
서울 조식 메뉴
학식 메뉴 확인할래요
나의 수업(e클래스) 강의자료를 확인하고싶어요
e클래스를 사용하고싶어요
졸업예정증명서
학생증 신청
서울 중식 메뉴
점심
wifi사용방법
서울 중식 메뉴
서울캠 내일 중식메뉴
학생지원팀
서울캠 내일 학식메뉴
중식 일품은 무슨 뜻이에요?
중앙사랑장학금
편의시설 안내
서울 중식 메뉴
학식 메뉴 확인할래요
라운지
중앙사랑
서울 중식 메뉴
미래융합라운지
학식 메뉴 확인할래요
교내 연락처 검색하고 싶어
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
경영학부 졸업시험
다빈치캠퍼스 주차장 이용 안내
졸업시험
주차장 이용에 대해 안내해주세요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치캠퍼스 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치 중식 메뉴
학식 메뉴 확인할래요
개인정보 수정하는 방법 알려줘
다빈치캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
다빈치 중식 메뉴
다빈치캠퍼스 학식메뉴
학생식당 위치
하계 계절학기
추가 학점
계절학기 수강신청은 언제/어떻게 하나요
다빈치캠퍼스 학식메뉴
도서관 위치
학생식당 위치
학식 메뉴 확인할래요
인문대학 교학지원팀 연락처 알려주세요
대학별 교학지원팀 연락처 알려주세요
서울 중식 메뉴
학식
단과대학별 공지사항 있나요?
경영학부 졸업시험 신청은 어디서 해?
서울 중식 메뉴
졸업하기 위해 필요한 요건이 어떻게 되나요?
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
석차
서울 중식 메뉴
강의시간표 확인하고 싶어요
학식 메뉴 확인할래요
이수변경신청
서울 중식 메뉴
도서관 이용 방법 알려주세요
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
e클래스를 사용하고싶어요
분실등록 어디서해?
학생증
학생증/신분증 발급내역 조회할 수 있나요
모바일 학생증/신분증 분실등록 및 기변신청은 어떻게 하나요
대학규정 및 학칙에 대해 알려주세요
레인보우시스템은 어떤 시스템인가요?
인문대학 교학지원팀 연락처 알려주세요
교내 건강센터
교내 연락처 검색하고 싶어
서울 중식 메뉴
교내 약국
학식 메뉴 확인할래요
강신청은 언제, 어떻게 하나요?
교내 연락처 검색하고 싶어
자퇴 신청은 어떻게 하나요
어떤 복지시설(편의시설)이 있나요?
휴학 신청 후 승인까지 얼마나 걸리낭ㅅ
서울 중식 메뉴
개인마당
학식 메뉴 확인할래요
강의계획서를 확인하고 싶어요
이수 학점 확인하고 싶어요
핵심교양
스마트학생증 신규발급
대학원 학생증 발급
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
이수 학점 확인하고 싶어요
학식 메뉴 확인할래요
다빈치 내일 중식메뉴
다빈치캠 내일 학식메뉴
건강센터 연락처
학식 메뉴 확인할래요
인바디 측정
인바디
예술공학대학 교학지원팀 연락처 알려주세요
e-Ad 메뉴_e-Advisor 설명
서울 중식 메뉴
학생생활상담센터 연락처
상담센터
학식 메뉴 확인할래요
단과대학 교학지원팀 연락처
영어 졸업요건 알려줘
공대 졸업요건 영어 알려줘
등록금 납부 방법에 대해서 알려주세요
학점등록은 언제/어떻게 하나요
소프트웨어대학 교학지원팀 연락처 알려주세요
e-Ad 메뉴_e-Advisor 설명
부서명
교내 연락처 검색하고 싶어
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
308관
학식 메뉴 확인할래요
전자학생증
전화번호 뭐야
교학지원팀 시간
교내 연락처 검색하고 싶어
휴학 후 계절학기
안내
경영경제대학 교학지원팀 연락처 알려주세요
부서명
서울캠 내일 중식메뉴
국제물류학과 사무실
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
부서명
교내 연락처 검색하고 싶어
과사무실
전화
학생증 재발급
구내서점
구내서점
편의시설 안내전화
안내전화
다빈치캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
서울캠퍼스 천원의아침밥 먹을래
교내 편의시설 안내
서점
학적부
수강신청
졸업 요건
ㅅㅜ강신청
304관
캠퍼스맵(건물 위치) 보여주세요
서울 중식 메뉴
학식 메뉴 확인할래요
일취월장장학금
재학생 장학금
일취
교직원식당
교수식당
서울 석식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
청강
경영경제대학 학사지원팀
지도
서울캠퍼스 주차장 이용 안내
주차
서울캠퍼스 천원의아침밥 먹을래
서울 중식 메뉴
학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
대학원 사물함
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
학생증 발급
서울 중식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
식당
융합전공
좌석 반납 안내
문화다양성
서울 중식 메뉴
열람실 좌석배정
학식 메뉴 확인할래요
서울 중식 메뉴
학식
도서관 이용 방법 알려주세요
서울 석식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
서울캠퍼스 학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
학생증 교부는 언제되나요?
서울 중식 메뉴
학생증 발급
학식 메뉴 확인할래요
서울 중식 메뉴
서울캠퍼스 학식 메뉴 확인할래요
장학금 수혜 내역서
모바일학생증 안내해주세요
학생증
서울 중식 메뉴
취업정보
서울 석식 메뉴
모바일서비스
서울 석식 메뉴
학식 메뉴 확인할래요
서울 조식 메뉴
서울 중식 메뉴
학식 가격
학식 갸격
학식 가갹
학생증 재발급
키드 재발급
사회과학대학 안내해주세요
단과대학 홈페이지
서울 중식 메뉴
학식 메뉴 확인할래요
융합전공 취소
모바일학생증 기변신청 방법
e id
서울 석식 메뉴
학식 메뉴 확인할래요
재학증명서 발급 안내해주세요
광고홍보학과 과 사무실
학비
증명서 발급 방법 알려주세요
모바일서비스
이수구분변경신청서
모바일 학생증 안드로이드
단과대학 교학지원팀 전화번호
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
재적증명서 발급 안내해주세요
이수구분 변경
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
제적증명서
학식 메뉴 확인할래요
다빈치 중식 메뉴
학식 메뉴 확인할래요
다빈치 중식 메뉴
학식 메뉴 확인할래요
열람실 좌석배정
도서관 이용 방법 알려주세요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
연구통합관리시스템(rss)가 무엇인가요
아이리스
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
학식
서울 중식 메뉴
학식 메뉴 확인할래요
재학생의 범위
재학생
김지우
교직원/교수님
서울 중식 메뉴
점심
서울 중식 메뉴
공과대학 교학지원팀 연락처 알려주세요
학식 메뉴 확인할래요
대학별 교학지원팀 연락처 알려주세요
강의실 대여할 수 있나요
모바일학생증 다운로드 방법
모바일 학생증/신분증이 뭔가요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
장학금 수혜
대학규정 및 학칙에 대해 알려주세요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
오늘 학식메뉴알려줘
카피킬러
서울 석식 메뉴
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
예비군 전입신고 방법은 무엇인가요
서울 중식 메뉴
예비군 소집훈련 안내
학식
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
등록금 금액 확인하고 싶어요
이수 학점 확인하고 싶어요
등록금
경영경제대학 교학지원팀 연락처 알려주세요
과사무실 연락처
등록금납입내역 확인할래요
등록금 금액(등록금 고지서) 확인하고 싶어요
구내서점
편의시설 안내
201관 위치
서울 중식 메뉴
학식 메뉴 확인할래요
서울캠퍼스 주차장 이용 안내
wnck
홍보팀 연락처
0원등록은 언제/어떻게 하는 건가요
서울 중식 메뉴
학식 메뉴 확인할래요
부서별 연락처안내
홍보팀
다빈치 중식 메뉴
학식 메뉴 확인할래요
자연과학대학 교학지원팀 연락처 알려주세요
교내 연락처 검색하고 싶어
학비납부
교내 연락처 검색하고 싶어
강의계획서
서울 중식 메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
학생식당 위치
중앙대학교 사업자등록번호
university club
사업자등록번호
학점
다빈치캠퍼스 석식 메뉴
나의 수업(e클래스) 과제를 확인하고싶어요
캠퍼스맵(건물 위치) 보여주세요
헬스장
e class
패스트푸드
서울 석식 메뉴
음식점 어디있어?
학식 메뉴 확인할래요
포탈 아이디/비밀번호를 잊어버렸어요
학교 와이파이 연결
수강신청
학생식당 위치
서울 중식 메뉴
어떤 복지시설(편의시설)이 있나요?
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치 중식 메뉴
서울 중식 메뉴
다빈치 조식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
체육대학 점심시간
UC CLUB
uc CLUB 예약
체육대학 입학처
서울 중식 메뉴
학식 메뉴 확인할래요
사범대학
시설팀
강의시간표 확인하고 싶어요
다빈치 중식 메뉴
수업시간표 알려줘
학식 메뉴 확인할래요
학적증명서
신입생 수강신청은 언제/어떻게 하나요
수강신청 가능학점은 몇 학점인가요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
독후감상문
적십자간호대학 교학지원팀 연락처 알려주세요
대학별 교학지원팀 연락처 알려주세요
강의실 대여할 수 있나요
서울 중식 메뉴
학식 메뉴 확인할래요
연계전공
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
점심 식당 마감은 언제가요?
학생회관
부서명
교내 연락처 검색하고 싶어
학술정보원(도서관) 모바일 앱이 있나요
수강신청은 언제, 어떻게 하나요?
e-Ad 메뉴_e-Advisor 설명
e클래스를 사용하고싶어요
구내서점
어떤 복지시설(편의시설)이 있나요?
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
미래설계장학금
서울 중식 메뉴
학식 메뉴 확인할래요
동일대체과목이란
중앙나래장학금
서울 생활관 우편 주소
서울 석식 메뉴
과목코드
복지장학금
생활관 우편
학식 메뉴 확인할래요
스마트학생증 재발급
todghkfrhks dnvus
스마트학생증 신규발급
중앙대 장학금
대학원생 학생증
마지막 학기에 6학점 듣는데 등록금은 얼마 내야해?
수강신청 정정
역량강화장학금
복학생 학생증 신청
서울 중식 메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
설계장학
중앙인
일취월장
학점등록은 언제/어떻게 하나요
서울 중식 메뉴
학식 메뉴 확인할래요
강의계획서를 확인하고 싶어요
교내 연락처 검색하고 싶어
월별 학사 일정이 궁금해요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
학식 메뉴 확인할래요
다빈치 중식 메뉴
국어국문학과 과사무실
학식 메뉴 확인할래요
이수 학점 확인하고 싶어요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
학생증 발급
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
수강신청
학식
학섹
zoom
수강신청 과목종류
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
비교과 프로그램은 어떤게 있나요?
내가 신청한 비교과 프로그램 일정을 알려주세요
서울 중식 메뉴
다빈치 중식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
자연과학대락
레인보우 문의
 비교과 장학금
학생지원팀 연락처
서울 중식 메뉴
인권센터 안내
학식 메뉴 확인할래요
4월 학사일정
월별 학사 일정이 궁금해요
다빈치 중식 메뉴
2024년 1학기 중간고사 기간
서울 중식 메뉴
생명공학대학 교학지원팀
학식 메뉴 확인할래요
학식 메뉴 확인할래요
중간고사 기간
재학생 장학금
장학금
경영경제대학 교학지원팀 연락처 알려주세요
경영경제대학 안내해주세요
학과사무실 전화
단과대학 홈페이지
강의계획서를 확인하고 싶어요
부전공취소
융합전공
이수 학점 확인하고 싶어요
소프트웨어인문융합전공
zoom
연계전공 신청
서울 중식 메뉴
학점등록
다빈치 석식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
연계전공
인문대학 교학지원팀 연락처 알려주세요
융합전공
서울캠퍼스 주차장 이용 안내
주차관리실 전화번호
휴학에 대해 알려주세요
교내 연락처 검색하고 싶어
e클래스를 사용하고싶어요
등록금고지서 어디 있어요?
서울 중식 메뉴
학식 메뉴 확인할래요
지도
상담센터
서울 중식 메뉴
점심
상담 일정 확인하고 싶어요
폐강
다빈치 내일 중식메뉴
학생증 재발급
다빈치캠 내일 학식메뉴
등록금고지서
서울 석식 메뉴
학생증 수령
학식 메뉴 확인할래요
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치 학생증 교부
서울 중식 메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
이수 학점 확인하고 싶어요
학생증 교부는 언제되나요
기기변경
휴학
동물생명공학
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학생증/신분증 발급내역 조회할 수 있나요
서울 중식 메뉴
경영경제대학 교학지원팀 연락처 알려주세요
학과사무실
학식 메뉴 확인할래요
분관상호대차
도서관 이용 방법 알려주세요
등록금 금액(등록금 고지서) 확인하고 싶어요
학생증 교부는 언제되나요?
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치 중식 메뉴
서울캠퍼스 학식 메뉴 확인할래요
학식 메뉴 확인할래요
학점등록
서울 중식 메뉴
학식 메뉴 확인할래요
어떤 복지시설(편의시설)이 있나요?
학생증발급
증명서 발급 방법 알려주세요
다빈치캠퍼스 천원의아침밥 먹을래
e클래스를 사용하고싶어요
서울 중식 메뉴
소프트웨어대학 교학지원팀 연락처 알려주세요
입학에 대해 궁금한게 있어요
학식
교내 연락처 검색하고 싶어
요람
대학원 요람
스마트학생증 신규발급
학생증
서울 중식 메뉴
학식 메뉴 확인할래요
학생증 발급
강의계획서
졸업논문제출자격시험
강의실 위치
서울 중식 메뉴
610관
학식 메뉴 확인할래요
안전교육
서울 중식 메뉴
학식 메뉴 확인할래요
산업안전교육
우체국
서울 중식 메뉴
학생증발급
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
교내 토익
모바일학생증 다운로드 방법
교내 토익 사라졌나요?
usim 학생증
서울캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
서울 중식 메뉴
학식 메뉴 확인할래요
이수체계도 확인하기
서울 중식 메뉴
gkrtlr
대학원 선수과목 인정
월별 학사 일정이 궁금해요
서울 중식 메뉴
학식 메뉴 확인할래요
교내 연락처 검색하고 싶어
참슬기
도시시스템 강의실
서울 중식 메뉴
학식 메뉴 확인할래요
이수 학점 확인하고 싶어요
서울 중식 메뉴
학식 메뉴 확인할래요
국제 처 시간
재학증명서
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치 중식 메뉴
서울 중식 메뉴
학식
학식 메뉴 확인할래요
mro
서울 중식 메뉴
의무실
ㅡ개
셔틀
학식 메뉴 확인할래요
학식 메뉴 확인할래요
이메일
학사 메일
가족장학금
재학생장학금
서울 중식 메뉴
학식 메뉴 확인할래요
서울캠퍼스 천원의아침밥 먹을래
교내 연락처 검색하고 싶어
서울 중식 메뉴
가족장학 신청
학식 메뉴 확인할래요
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치캠퍼스 석식 메뉴
usim
교내 연락처 검색하고 싶어
다빈치 중식 메뉴
서울 중식 메뉴
서울캠퍼스 학식 메뉴 확인할래요
학식
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치 810
공인인증서
등록 기간
인증서발급
편의점
서울 중식 메뉴
학식 메뉴 확인할래요
범용인증서
다빈치 캠퍼스 강의실 위치
다빈치캠퍼스
서울 중식 메뉴
학식 메뉴 확인할래요
백분위
다빈치 캠퍼스 위치
다빈치캠퍼스 맵
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
중앙사랑 장학금
학식 메뉴 확인할래요
다빈치 캠퍼스 위치
서울 중식 메뉴
학식 메뉴 확인할래요
월별 학사 일정이 궁금해요
서울 중식 메뉴
창아공
일취월장 장학금
학식 메뉴 확인할래요
연계전공
중앙사랑 장학금
신입생 장학금
서울 중식 메뉴
학식 메뉴 확인할래요
mro
다빈치캠퍼스 석식 메뉴
mro
다빈치 중식 메뉴
온라인 폭력예방교육이 뭐야
서울 중식 메뉴
학식 메뉴 확인할래요
체육대학 입학처
eid 재발급
전공심화란 무엇인가요
서울 석식 메뉴
일취월장장학금
서울 중식 메뉴
학식 메뉴 확인할래요
재학생장학금
성적증명서
총무처 연락처
서울 중식 메뉴
학식 메뉴 확인할래요
중앙사랑장학금
성적우수장학금
전화번호 뭐야
재학생 장학금
장학금정보가 궁금해요
나 지금 몇학점 들어?
학번
자연대 교학지원팀
서울 중식 메뉴
조기졸업 학점 다 채워야해?
내 학점
국제처 시간
학식 메뉴 확인할래요
서울 중식 메뉴
단과대학 교학지원팀 연락처
학식 메뉴 확인할래요
국제 처
강의시간표 확인하고 싶어요
조기졸업 석사요건
서울캠 내일 중식메뉴
나의 도서관 이용현황이 궁금해요
서울캠 내일 학식메뉴
무인도서대출
서울 중식 메뉴
한자 몇 급 따야 졸업할 수 있어?
학식 메뉴 확인할래요
스마트학생증 재발급
학생증
월별 학사 일정이 궁금해요
분실등록 어디서해?
학생증
연계전공 신청
연계전공
행정
학생증/신분증 발급내역 조회할 수 있나요
부서명
졸업을 위한 논문, 시험, 작품에 대해서 알려주세요
서울 중식 메뉴
이수변경신청
등록금 부서
졸업하기 위해 필요한 요건이 어떻게 되나요?
24년 예비군
서울 중식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
선수과목
이수 학점 확인하고 싶어요
생명공학대학 교학지원팀 연락처 알려주세요
융합전공 이수 구분
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
예비군 소집훈련 안내
교내 연락처 검색하고 싶어
학식 메뉴 확인할래요
학생식당 위치
졸업 과정
학식 메뉴 확인할래요
장학금 부서
대학 및 학과(부)별 졸업이수학점 알려주세요
서울 중식 메뉴
학식 메뉴 확인할래요
이수구분변경신청서
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치 중식 메뉴
인문대 교학지원팀
다빈치캠퍼스 천원의아침밥 먹을래
편의시설 안내
학식 메뉴 확인할래요
강의실 위치
다빈치 위치
다빈치 위체
세부전공 변경
다빈치 캠퍼스 지도
경제학원론 수강한 경영학부 학생이 전공 인정 신청 어떻게해
다빈치 캠퍼스 매
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
강의실 위치
서점
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
자소서 첨삭
학식 메뉴 확인할래요
과목번호
과목번호
전공 변경
졸업 요건
전공변경
서울 중식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
바보
서울 조식 메뉴
공과대학 교학지원팀 연락처 알려주세요
와이파이
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
부서명
아고라존
서울 중식 메뉴
학식 메뉴 확인할래요
교내 연락처 검색하고 싶어
졸업요건
서울 석식 메뉴
학식 메뉴 확인할래요
아이림스
02-820-5248
서울 중식 메뉴
서울 중식 메뉴
다전공 변경
학식 메뉴 확인할래요
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
공과대학 교학지원팀 연락처 알려주세요
대학원 교학지원팀 연락처
평균학점
공과대학 교학지원팀 연락처 알려주세요
다빈치 중식 메뉴
교내 연락처 검색하고 싶어
학식 메뉴 확인할래요
일취월장장학금
재학생장학금
서울 석식 메뉴
일취월장
학식 메뉴 확인할래요
강의시간표 확인하고 싶어요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
성적증명서 발급할래요
기숙사
안성캠 기숙사 출입 방법
폐강
내 지도교수님
서울 석식 메뉴
재적증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
의과대학 전공
학생지원팀
서울 중식 메뉴
학식 메뉴 확인할래요
우리은행은 어디있나요?
서울 중식 메뉴
학생처
학식 메뉴 확인할래요
스마트학생증 신규발급
가족장학금
성적증명서 발급할래요
이수 학점 확인하고 싶어요
학생증 재발급
학생증
재학생 장학금
장학금
나 졸업하려면 어떻게 해야 돼?
증명서 발급 방법 알려주세요
모바일 학생증/신분증 어떻게 다운로드 하나요
조기졸업
예술대학 교학지원팀 연락처 알려주세요
학생증 분실
온라인 폭력예방교육이 뭐야
전화번호 뭐야
교내 행정부서의 근무시간을 알려주세요
모바일학생증
막학기 등록금
전화번호 뭐야
교내 행정부서의 근무시간을 알려주세요
교양대학 교학지원팀
교양대학 상담시간
다빈치 캠퍼스 행정팀
이수 학점 확인하고 싶어요
부서별 위치 및 전화번호
수강한 과목의 이수 구분을 변경하고 싶어요
교양대학 교학지원팀
교양다학
도서관 사물함
성적증명서 발급할래요
교내 연락처 검색하고 싶어
도서고ㅓㄴ ㅔ
부서별 위치 및 전화번호
도서고ㅓㄴ
전과 학점 인정
강의계획서 입력
학생증 분실
캠퍼스맵(건물 위치) 보여주세요
졸업하기 위해 필요한 요건이 어떻게 되나요?
등록금 관련 문의는 어떤 부서로하나요?
성적조회
강의평가는 언제 어떻게 진행하나요?
서울 중식 메뉴
학식 메뉴 확인할래요
열람실 좌석배정
도서관 이용 방법 알려주세요
교양대학 교학지원팀
국가근로 중단
교양대학
경영경제대학 교학지원팀
경영경제대학 전화번호 알려줘
교내 연락처 검색하고 싶어
학생식당 위치
법식
법학관 식당
캠퍼스맵(건물 위치) 보여주세요
최소학점
서울 중식 메뉴
서울캠퍼스 학식메뉴 알려줘
서울 중식 메뉴
학식 메뉴 확인할래요
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
등록금 반횐
학생증 재발급
서울 석식 메뉴
이수 학점 확인하고 싶어요
휴학
서울 중식 메뉴
학식 메뉴 확인할래요
사회학과
서울 중식 메뉴
학식
서울 중식 메뉴
학식 메뉴 확인할래요
학점교류
학생증 분실
서울 중식 메뉴
학식 메뉴 확인할래요
모바일 학생증/신분증 분실등록 및 기변신청은 어떻게 하나요
교내 부서(팀)별 연락처 알려주세요
재학증명서 팩스
서울 중식 메뉴
근로
학식 메뉴 확인할래요
학적부 팩스
email
portal
서울 중식 메뉴
나노바이오소재 학과사무실
서울 중식 메뉴
학식 메뉴 확인할래요
등록기간
행정처
서울 중식 메뉴
교내 연락처 검색하고 싶어
학식 메뉴 확인할래요
학식 메뉴 확인할래요
등록금 납부 관련 담당부서
교내 연락처 검색하고 싶어
다빈치 중식 메뉴
학식 메뉴 확인할래요
등록금
등록금 납부 관련 문의
다빈치 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
사회복지대학원 사무실
등록금 납부 방법에 대해서 알려주세요
다빈치 중식 메뉴
학식 메뉴 확인할래요
강의계획서를 확인하고 싶어요
서울 중식 메뉴
학식
서울 중식 메뉴
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치 중식 메뉴
학식 메뉴 확인할래요
논문자격시험
서울 중식 메뉴
학식 메뉴 확인할래요
다른대학 교학지원팀 연락처
서울 중식 메뉴
학식 메뉴 확인할래요
생명공학대학 교학지원팀 연락처 알려주세요
레인보우시스템은 어떤 시스템인가요?
0316703391
남궁혜경
교직원/교수님
이수 학점 확인하고 싶어요
등록금
교내 연락처 검색하고 싶어
서울 중식 메뉴
학식 메뉴 확인할래요
서울 석식 메뉴
예술대학 교학지원팀 연락처 알려주세요
월별 학사 일정이 궁금해요
학생지원팀 연락처
대학원 교학지원팀 연락처
학과사무실
장학
외부장학
부서명
교내 연락처 검색하고 싶어
서울 중식 메뉴
학식 메뉴 확인할래요
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
서점
한자 자격증
행정팀
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
행정
캠퍼스맵(건물 위치) 보여주세요
실험방
전공심화
글로벌금융학과 전화번호
학생증/신분증 사진 변경 방법 알아보기
F맞으면 재수강
연계전공
도서관 이용 방법 알려주세요
휴학취소
다빈치캠 학생지원팀 번호
감사합니다
우리은행 위치
중앙대학교 안에 우리은행이 어디있나요?
이아이디
모바일학생증 기변신청 방법
이아이디
경영학부 사무실
학점등록
한자
I don't know wherebis my class room
생명공학대학 교학지원팀 연락처 알려주세요
e-Ad 메뉴_e-Advisor 설명
단과대학 교학지원팀 연락처 알려주세요
필수로 이수해야 하는 과목은 어떤 과목인가요
강의시간표 확인하고 싶어요
학적부
월별 학사 일정이 궁금해요
경영경제대학 교학지원팀 연락처 알려주세요
학생 학과 사무실
eid 사진
전화번호 뭐야
스마트학생증 재발급
종합정보시스템
학샌증
교양다학 행정실
경영학과 사무실
대학 및 학과(부)별 졸업이수학점 알려주세요
강의계획서를 확인하고 싶어요
교양대학
경영대 교학지원팀
종합정보시스템
교양학부
경영대 교학지원팀
부서명
실험실 사용
교내 연락처 검색하고 싶어
강의계획서를 확인하고 싶어요
장학내역조회
서울캠 내일 석식메뉴
경영대학 학과사무실
와이파이 사용방법이 궁금해요
나의 수업(e클래스) 강의자료를 확인하고싶어요
e클래스를 사용하고싶어요
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
출퇴근 기록
국제교류
모바일 학생증/신분증 분실등록 및 기변신청은 어떻게 하나요
스마트학생증 신규발급
출근
외국인유학생에 대해 알려주세요
학생증
생명공학대학 교학지원팀
공학인증 확인
기기변경
절대평가 변환기준
교양대학 교학지원팀
다빈치교양
절대평가
대학별 교학지원팀 연락처 알려주세요
회의실
대실
서울캠퍼스 주차장 이용 안내
재학생 차량 등록은 어떻게 하나요?
비밀번호 변경
교육학과 학과사무실 번호
내 정보 변경
서울 중식 메뉴
학식 메뉴 확인할래요
한자
서울 중식 메뉴
학식 메뉴 확인할래요
공학인증
와이파이 사용방법이 궁금해요
학생지원팀
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
공학인증
학교 지도
서울 중식 메뉴
생명공학대학 교학지원팀 연락처 알려주세요
학식 메뉴 확인할래요
단과대학 교학지원팀 연락처
일반대학원교육지원팀
예비군 전입신고 방법은 무엇인가요
대학원 교학지원팀 연락처
안성캠퍼스 생명공학대학 교학지원팀
행정실 연락처
서울 중식 메뉴
휴학
학식 메뉴 확인할래요
서울캠 내일 중식메뉴
서울캠퍼스 학식메뉴
학생식당
801
서울 중식 메뉴
학식 메뉴 확인할래요
2024 4월 학사일정
스마트학생증 신규발급
서울캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
와이파이 사용방법이 궁금해요
2024 1학기 중간고사
학생증
와이파이 사용방법이 궁금해요
중간고사
서울 중식 메뉴
학식 메뉴 확인할래요
지도
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
나노바이오소재 학과사무실 번호
성폭력예방 교육
서울 중식 메뉴
학식 메뉴 확인할래요
예비군
서울 중식 메뉴
학생증 분실신고 했는데 찾았어요
서울 조식 메뉴
서울 중식 메뉴
다빈치캠퍼스 날씨
재학증명
서울캠 내일 학식메뉴
학식
교육비납입증명서 발급 안내해주세요
날씨 알려줘
8학기 등록금
등록금 납입확인서 발급 안내해주세요
학생식당 위치
교육학과
증명서 발급 방법 알려주세요
학식 메뉴 확인할래요
등록금 납부기간
졸업 등록금
등록금납입내역 확인할래요
취업정보
학자금 영수증
서울 중식 메뉴
학식 메뉴 확인할래요
국문과
스마트학생증 신규발급
학생증
수강신청
사회과학대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀 연락처
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
종합정보시스템
학점등록은 언제/어떻게 하나요
종합정보시스템
등록금
다빈치캠퍼스 석식 메뉴
학생지원팀 연락처
다빈치 교학지원팀
졸업요건
학생증 재발급
장학지원팀
서울 중식 메뉴
장학지원
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
학식 메뉴 확인할래요
동아리
서울캠퍼스 스터디룸 예약
팀플룸
서울 중식 메뉴
학식 메뉴 확인할래요
zoom
서울 중식 메뉴
학생증 사진변경
학식 메뉴 확인할래요
학생증 재발급
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치 중식 메뉴
학교 셔틀
학식 메뉴 확인할래요
서울 석식 메뉴
한자
졸업 요건
대학원 모집일정을 보고싶어요
셔틀
서울 중식 메뉴
학식 언제 열려
학석연계과정
서울 중식 메뉴
체육교육과
학식
동일대체과목이란
내 정보
서울 중식 메뉴
성적증명서 발급 안내해주세요
학식 메뉴 확인할래요
증명서 발급 방법 알려주세요
수업계획서
경제학부
irims
등록금 납부 방법에 대해서 알려주세요
대학규정 및 학칙에 대해 알려주세요
입학에 대해 궁금한게 있어요
이수 학점 확인하고 싶어요
서울 중식 메뉴
식당
서울 중식 메뉴
학식 메뉴 확인할래요
동일대체과목이란
경영경제대학 교학지원팀 연락처 알려주세요
학과사무실
서울 중식 메뉴
학사경고 후 복학
학사경고
서울 석식 메뉴
학사탐
학식 메뉴 확인할래요
강신청은 언제, 어떻게 하나요?
부서별 연락처안내
학생지원팀
학사팀
단과대 교학지원팀 전화번호 알려줘
이수학점
조기졸업(우등졸업)은 누가/어떻게 할 수 있나요
부서별 연락처안내
졸업유예
서울 중식 메뉴
학식 메뉴 확인할래요
학생지원팀
사전장학
서울 중식 메뉴
학식 메뉴 확인할래요
체육대학 교학지원팀
서울 중식 메뉴
학식 메뉴 확인할래요
이수 학점 확인하고 싶어요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
추가등록
잔여 휴학
선수과목
사용가능한 휴학
추가등록
서울 중식 메뉴
학생증 재발급
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
학생증 재발급
휴학 신청됐는지 어떻게 확인해?
다빈치 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
성적우수장학금
재학생장학금
서울 석식 메뉴
교환학생장학금
서울캠 내일 석식메뉴
서울캠 내일 중식메뉴
재학생 장학금
캠퍼스맵(건물 위치) 보여주세요
서울캠 내일 학식메뉴
장학금 정보가 궁금해요
졸업하기 위해 필요한 요건이 어떻게 되나요?
서울 중식 메뉴
학식 메뉴 확인할래요
선수과목 교칙
다빈치 셔틀
카우버거 가격
졸업하기 위해 필요한 요건이 어떻게 되나요
인문대학
소프트웨어대학 교학지원팀
서울 중식 메뉴
소프트웨어대학 알고리즘 선수과목
카우버거
0원등록
학생증 재발급
e비즈니스
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
e비즈니스 강의실
인사팀
군입대 휴학 및 복학은 어떻게 신청하나요
강의시간표 확인하고 싶어요
강의평가는 언제 어떻게 진행하나요?
동물생명공학
인권센터 안내
서울 중식 메뉴
학식 메뉴 확인할래요
시설팀
유연학기제
교내셔틀
체육대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀 연락처 알려주세요
전공실기
E상담센터
재학증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
등록금 납부 기한
증명서 자판기
서울 중식 메뉴
학식
학적부 발급 안내해주세요
증명서 발급 방법 알려주세요
스마트학생증 재발급
학적부 발급 안내해주세요
성적증명서 발급 안내해주세요
학생증 신청
증명서 발급 방법 알려주세요
서울 석식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
대학원생 사물함
학점등록
사물함 신청(도서관/학과) 어떻게 해
추가등록기간
지도
국제교류팀
부서명
교내 연락처 검색하고 싶어
국제교류팀
교직원/교수님
교내 연락처 검색하고 싶어
학점등록
다빈치 중식 메뉴
등록금 내기
학식 메뉴 확인할래요
국제교류팀
교내 연락처 검색하고 싶어
추가등록 하는 법
서울 중식 메뉴
학식 메뉴 확인할래요
이수체계도 확인하기
추가등록 하는 법
전공필수
서라벌홀 가는법
교내 연락처 검색하고 싶어
서울 중식 메뉴
교학지원팀 위치
휴학신청 취소
캠퍼스맵(건물위치) 보여주세요
사회과학대학 교학지원팀 연락처 알려주세요
서울캠퍼스 천원의아침밥 먹을래
부서별 위치 및 전화번호
단과대학 교학지원팀 연락처
캠퍼스맵(건물 위치) 보여주세요
학식 메뉴 확인할래요
부서별 위치 및 전화번호
예술대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀 연락처
일취월장
캠퍼스맵(건물위치) 보여주세요
와이파이 사용방법이 궁금해요
국문과
학점등록
스마트학생증 재발급
학생증
2학년 편입
일취월장
학점등록
장학금 수혜확인서
학점등록
서울 중식 메뉴
학식 메뉴 확인할래요
행정 번호
학적부 발급
번호
백분위성적
행정실
교직과목이수확인서 발급 안내해주세요
셔트
학식 메뉴 확인할래요
창업경영대학원
창업지원팀
교내 연락처 검색하고 싶어
예비군연대
다빈치캠퍼스 예비군
다빈치캠퍼스
다빈치연댜ㅐ
이수 학점 확인하고 싶어요
서울 중식 메뉴
학식 메뉴 확인할래요
셔틀
스마트학생증 재발급
학생증
서울 중식 메뉴
학적부 발급 안내해주세요
학식 메뉴 확인할래요
I lost my student card
복수전공
학적부 발급 안내해주세요
독후감
토익
졸업사정
사회과학대학 교학지원팀
증명서 발급 방법 알려주세요
사회과학대학 안내해주세요
인문대학교학지원팀
단과대학 홈페이지
창업지원팀
부전공
학생식당 위치
유연학기제
어떤 복지시설(편의시설)이 있나요?
신입생 시간표
교내 연락처 검색하고 싶어
학생식당 위치
석식
pc
석사
부서명
토익
교내 연락처 검색하고 싶어
졸업요건
사회과학대학 교학지원팀 연락처 알려주세요
교내 연락처 검색하고 싶어
교내 연락처 검색하고 싶어
선수과목
학점등록은 언제/어떻게 하나요
학점등록
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
복사실
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
졸업증명서 영문 출력
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
사회과학대학 교학지원팀 연락처 알려주세요
도시계획부동산학과 사무실 연락처
재난복구센터
학식 시간
모바일학생증 기변신청 방법
모바일학생증 다운로드 방법
서울 중식 메뉴
학식 메뉴 확인할래요
eid
모바일학생증 발급방법
기기변경
월별 학사 일정이 궁금해요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
시발
서울 중식 메뉴
다빈치캠 캠퍼스맵
휴학에 대해 알려주세요
독후감
교수님 수업
dr센터
동일한 교수님 수어
서울 중식 메뉴
같은 교수님 수업
추가학기
같은 교수님
식당
다빈치캠 DR
dr
인재개발팀
경영학과 사무실 연라처
교내 연락처 검색하고 싶어
안전정보센터
ebook
인재개발팀
학생지원
나의 수업(e클래스) 과제를 확인하고싶어요
인문학40선
e클래스를 사용하고싶어요
등록금 납부기간
바이오메디컬
EHS
스마트학생증 재발급
학점등록은 언제/어떻게 하나요
기숙사
학생증발급
조교
서울캠퍼스 주차장 이용 안내
주차
학점등록
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학교 와이파이
학생증/신분증 사진 변경 방법 알아보기
출석부 사진
재학증명
서울 중식 메뉴
학식 메뉴 확인할래요
월별 학사 일정이 궁금해요
서울 중식 메뉴
졸업 관련 증명서 발급 안내해주세요
서울캠퍼스 학식메뉴 알려줘
증명서 발급 방법 알려주세요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
다빈치 조식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
서울 조식 메뉴
학식
강의계획서를 확인하고 싶어요
중앙인
학점등록
인문대학교학지원팀
유연학기제가 어떤건지 알려줘
중앙인 게시판
서울 생활관에 대한 정보는 어디서 보나요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
학교 대처방안
캠퍼스맵(건물 위치) 보여주세요
코로나19 확진자 현황 알려주세요
상대평가
다빈치 러닝 수업
다빈치러닝이 무엇인가요
학교 축구장 사용
재수강 최대학점
우편물실
서울캠 내일 석식메뉴
서울캠 내일 학식메뉴
졸업증명서
증명서 발급 방법 알려주세요
서울 석식 메뉴
학식 메뉴 확인할래요
부서별 위치 및 전화번호
편의시설 안내
학교 축구장
동아리
다빈치 셔틀
term2
비교과 프로그램은 어떤게 있나요?
서울 중식 메뉴
학식 메뉴 확인할래요
레인보우시스템은 어떤 시스템인가요?
유연학기제가 어떤건지 알려줘
다빈치 셔틀
대학규정 및 학칙에 대해 알려주세요
등록금납입내역 확인할래요
다빈치러닝
등록금
서울캠 내일 석식메뉴
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 석식 메뉴
서점
기기변경
학부에 대해 안내해주세요
서울캠퍼스 학과
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
캠간버스
와이파이
와이팡
모바일학생증 다운로드 방법
다빈치캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
모바일학생증 기변신청 방법
모바일학생증 발급방법
모바일 학생증/신분증 분실등록 및 기변신청은 어떻게 하나요
추가 학기
교내 연락처 검색하고 싶어
등록금납입확인서
다빈치 조식 메뉴
학식 메뉴 확인할래요
강의계획서를 확인하고 싶어요
이수 학점 확인하고 싶어요
서울 중식 메뉴
학식 메뉴 확인할래요
인적자원관리 선수과목
선수과목
서울 중식 메뉴
서울 석식 메뉴
학식 메뉴 확인할래요
강의노트
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
나의 수업(e클래스) 과제를 확인하고싶어요
대학원지원팀 연락처
e클래스를 사용하고싶어요
대학원 24년1학기 수강신청기간
이수학점
수강신청기간
졸업학점
나의 수업(e클래스) 과제를 확인하고싶어요
복수전공
e클래스를 사용하고싶어요
와이파이
교내 인터넷
지도
등록은 언제, 어떻게 하나요?
환불계좌등록
강의계획서를 확인하고 싶어요
서울캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
재학증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
인문대학 교학지원팀 연락처 알려주세요
대학별 교학지원팀 연락처 알려주세요
역사학과 교학지원팀
재학증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
서울 중식 메뉴
학식 메뉴 확인할래요
캠퍼스맵(건물 위치) 보여주세요
다빈치 조식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
서울 조식 메뉴
학식 메뉴 확인할래요
서울캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
서울 조식 메뉴
학식 메뉴 확인할래요
서울캠퍼스 천원의아침밥 먹을래
중앙대 셔틀
중앙대 교내셔틀
서울 조식 메뉴
학식 메뉴 확인할래요
학생증 재발급
스마트학생증 재발급
학생증 발급
셔틀
수강정정
캠퍼스맵(건물위치) 보여주세요
교내 셔틀
서울 중식 메뉴
학식 메뉴 확인할래요
이수 학점 확인하고 싶어요
진도점검
서울 중식 메뉴
서울캠퍼스 천원의아침밥 먹을래
서울 조식 메뉴
학식 메뉴 확인할래요
생활관 스터디룸
학생증 사진 변경 기간이 언젠가요
학생증 사진 변경 기강 언젠가요
학식 메뉴 확인할래요
응용통계학과 사무실
와이파이 사용방법이 궁금해요
다빈치캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
카우버거 시간
다빈치 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
다빈치캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
최재오
교직원/교수님
교내 연락처 검색하고 싶어
교내 연락처 검색하고 싶어
서울 중식 메뉴
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
인문대학 안내해주세요
단과대학 홈페이지
다빈치캠퍼스 석식 메뉴
다빈치 중식 메뉴
학식 메뉴 확인할래요
융합전공 취소
강의계획서
해외교류프로그램 장학금에 대해 알고싶어요
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
해외인턴
등록금납입내역 확인할래요
등록금 납부 방법에 대해서 알려주세요
서울캠퍼스 천원의아침밥 먹을래
천원의
정정
통학버스
서울 중식 메뉴
학식 메뉴 확인할래요
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
다빈치캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
서울 조식 메뉴
학식 메뉴 확인할래요
월별 학사 일정이 궁금해요
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
셔틀버스 시간표와 노선을 알고 싶어요
졸업을 위한 논문, 시험, 작품에 대해서 알려주세요
서울캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
졸업인정제(영어/한자)에 대해서 알려주세요
서울 조식 메뉴
308관 식당
다빈치캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
서울 중식 메뉴
학식 메뉴 확인할래요
기념품판매점
서울 중식 메뉴
학식 메뉴 확인할래요
어떤 복지시설(편의시설)이 있나요?
주말 개방
졸업하기 위해 필요한 요건이 어떻게 되나요?
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
졸업 논문 양식
단과대학별 공지사항 있나요?
졸업 논문 양식 파일 어디있엎
다빈치 조식 메뉴
학식 메뉴 확인할래요
학위 논문 양식 알려줘
서울 조식 메뉴
학식 메뉴 확인할래요
통학
서울 중식 메뉴
학식 메뉴 확인할래요
강의계획서를 확인하고 싶어요
강의시간표 확인하고 싶어요
강의계획서를 확인하고 싶어요
졸업하기 위해 필요한 요건이 어떻게 되나요?
졸업학점
등록금 납부 방법에 대해서 알려주세요
서울 중식 메뉴
학식 메뉴 확인할래요
Eid 재발급
서울 중식 메뉴
학식 메뉴 확인할래요
서울 조식 메뉴
학식 메뉴 확인할래요
성적증명서
학생증신청
서울캠퍼스 천원의아침밥 먹을래
학식
다빈치 중식 메뉴
월별 학사 일정이 궁금해요
학식 메뉴 확인할래요
수강신청
비대면수업 및 대면수업 일정 안내
도서관 시설 안내
서울도서관 운영시간
도서관 출입구
310관 개방시간
개방시간
이클래스 강의자료 확인하기
나의 강의교재 확인하기
나의 수업(e클래스) 강의자료를 확인하고싶어요
e클래스를 사용하고싶어요
다빈치도서관 운영시간
다빈치도서관 운영시간
다빈치도서관 운영시간
도서관 시간
학교 개방시간
개방시간
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
병결
어떤 경우에 출석인정이 가능한가요
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
학생증 교부는 언제되나요?
학생증 신청
유연학기제가 어떤건지 알려줘
카드학생증방급
스마트학생증 재발급
학생증발급
서울 중식 메뉴
서울캠퍼스 천원의아침밥 먹을래
학식
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
도서관 이용 방법 알려주세요
7교시
등록금 추가
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
사물함 신청(도서관/학과) 어떻게 해
개인 사물함이 있어?
캠간셔틀
월별 학사 일정이 궁금해요
강의계획서
서울 중식 메뉴
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
온라인 서점
구내서점
교내 편의시설 안내
중앙대 서점
연계전공
공기업관리
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
가족장학금신청
장학금정보가 궁금해요
가족장학금
재학생 장학금
장학금정보가 궁금해요
복수전공
수강신청마감버튼
마감버튼
수강과목취소
수강신청마감
수강신청 정정
강의계획서
강의평가는 언제 어떻게 진행하나요?
출석인정제 신청하고 싶어요
결석사유
수강신청 삭제가 안돼요
중국어문학과
교환학생 및 방문학생에 대해 알려주세요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 석식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
성적증명서
대학원 졸업학점 조회
대학원 졸업학점 조회
학점조회
등록금 납부 방법에 대해서 알려주세요
졸업하기 위해 필요한 요건이 어떻게 되나요?
졸업 가능여부(졸업사정)를 확인하고 싶어요
복수전공
추가학기
학석 연계
계절학기 수강신청은 언제/어떻게 하나요
학생식당 위치
점심
재수강 인정
수강한 과목의 이수 구분을 변경하고 싶어요
졸업
졸업요건
학점등록+재수강
학점등록
편입생 학점인정을 안내해주세요
최소이수
예술공학대학 교학지원팀 연락처 알려주세요
전공기초
이수구분 변경 전공기초?
단과대학 교학지원팀 연락처
이수구분변경 가능해?
학점등록
다빈치캠퍼스 석식 메뉴
다빈치 중식 메뉴
학식 메뉴 확인할래요
성적우수 기준
장학내역 발급
입학날짜 학인
학생생활상담
입학/졸업 일자를 확인하고 싶어요
학생생활상담센터
학점등록
서울캠 내일 조식메뉴
다빈치 석식 메뉴
학식 메뉴 확인할래요
학생식당 위치
서울캠 내일 학식메뉴
다빈치캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
서울캠퍼스 천원의아침밥 먹을래
입학/졸업 일자를 확인하고 싶어요
학식 메뉴 확인할래요
학생증/신분증 발급내역 조회할 수 있나요
입학날짜
다빈치 중식 메뉴
다빈치 조식 메뉴
다빈치캠퍼스 학식 메뉴 확인할래요
학생지원팀 연락처
장학금 관련 문의
외부장학 공지
장학금 관련
학생증 수령
내 학과
내정보
경영경제대학 교학지원팀 연락처 알려주세요
단과대학 교학지원팀 전화번호
이수구분신청서
중앙사랑장학금
봉사장학금
재학생 장학금
경영경제대학 교학지원팀 연락처 알려주세요
장학금 정보가 궁금해요
교학지원팀
학점등록
경영경제대학 교학지원팀 연락처 알려주세요
다른대학 교학지원팀 연락처
공과대학 교학지원팀 연락처 알려주세요
다른대학 교학지원팀 연락처
경영경제대학 교학지원팀
졸업 관련 증명서 발급 안내해주세요
이수구분신청
재적증명서 발급 안내해주세요
증명서 발급 방법 알려주세요
이수구분
서울 학사팀
203관 어디에요
재학증명
졸업식
포트폴리오 이용방법에 대해 안내해주세요
졸업예정일
모바일학생증 다운로드 방법
e-id
전자 출결
7교시 시간
전자출결
나의 교과년도를 조회해줘
서울 중식 메뉴
학식 메뉴 확인할래요
단과대학 교학지원팀 연락처 알려주세요
필수로 이수해야 하는 과목은 어떤 과목인가요
강의계획서를 확인하고 싶어요
나의 수업(e클래스) 강의자료를 확인하고싶어요
eclass
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
강의계획서를 확인하고 싶어요
교내 편의시설 안내
카우버거
강의계획서를 확인하고 싶어요
e-id 발급
추가등록
서울 중식 메뉴
학식 메뉴 확인할래요
학생증 재발급
서울캠퍼스 천원의아침밥 먹을래
아침
학점신청
학점인정
서울 중식 메뉴
학식 메뉴 확인할래요
학석 연계과정
학생증 재발급
서울 중식 메뉴
학식 메뉴 확인할래요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
수료
휴학에 대해 알려주세요
다빈치 조식 메뉴
학식 메뉴 확인할래요
서울캠 내일 석식메뉴
셔틀버스 시간표와 노선을 알고 싶어요
학기 수업일 수
서울캠 내일 중식메뉴
수업일수
서울캠 내일 학식메뉴
학생식당 위치
학식 메뉴 확인할래요
군 제대
국어국문학과
비대면수업 및 대면수업 일정 안내
국어국문학과
서울 중식 메뉴
학식 메뉴 확인할래요
외부장학 공지
장학금
봉사활동을 학점으로 인정받을 수 있나요
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
복지장학금
서울캠퍼스 천원의아침밥 먹을래
서울 조식 메뉴
학식 메뉴 확인할래요
미래설계장학금
전공평점
앙트레 대체
회계학
중앙나래장학금
재학생 장학금
장학금 정보가 궁금해요
중앙대 서울캠퍼스 셔틀버스 얼마야?
중앙대 셔틀버스
증명서 발급 방법 알려주세요
국가 장학금
장학금 정보가 궁금해요
학생예비군
전과 문의
서울 중식 메뉴
학식 메뉴 확인할래요
신청 가능한 장학금이 있나요
cau grs 장학금이란 뭔가요
서울 중식 메뉴
학식 메뉴 확인할래요
중앙사랑장학금
아니 일취월장
일취월장 어떻게 신청해?
월별 학사 일정이 궁금해요
일취월장장학금
재학생 장학금
장학금 정보가 궁금해요
일취월장장학금
재학생 장학금
장학금 정보가 궁금해요
학생증/신분증 발급내역 조회할 수 있나요
다빈치캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
서울캠 내일 석식메뉴
서울캠 내일 중식메뉴
서울캠 내일 조식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
복사실
편의시설 안내
107관이 어디야
다빈치캠 셔틀
캠퍼스간셔틀버스
생활관
스마트학생증 신규발급
학생증
중앙사랑 장학금
캠간셔틀
월별 학사 일정이 궁금해요
신입생 장학금
장학금 정보가 궁금해요
서울 석식 메뉴
학식 메뉴 확인할래요
매트랩 설치
스마트학생증/신분증 신규발급 신청방법 알려주세요
재학증명서
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
학식 메뉴 확인할래요
서울 중식 메뉴
학식 메뉴 확인할래요
다빈치 내일 조식메뉴
다빈치캠퍼스 내일 학식메뉴
서울 석식 메뉴
다빈치 석식 메뉴
서울 중식 메뉴
서울 조식 메뉴
학식 메뉴 확인할래요
다빈치 중식 메뉴
다빈치 조식 메뉴
학식 메뉴 확인할래요
다빈치 중식 메뉴
학식 메뉴 확인할래요
신입생 장학금
장학금 정보가 궁금해요
도서관(학술정보원) 홈페이지 로그인 어떻게 하나요?
휴학신청
도서관
모바일 학생증/신분증 분실등록 및 기변신청은 어떻게 하나요
재수강
서울 중식 메뉴
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
등록금
이수 학점 확인하고 싶어요
학생증 발급
모바일학생증 안내해주세요
학생증
공과대학 교학지원팀 연락처 알려주세요
대학별 교학지원팀 연락처 알려주세요
서울캠 내일 조식메뉴
서울캠 내일 학식메뉴
서울 석식 메뉴
복전
학식 메뉴 확인할래요
재학증명서
체성분 분석 프로그램 신청은 어떻게 해?
서울 중식 메뉴
학식 메뉴 확인할래요
이수 학점 확인하고 싶어요
핵심교양
서울 석식 메뉴
서울 중식 메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
서울캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
조기졸업(우등졸업)은 누가/어떻게 할 수 있나요
중앙사랑장학금
재학생 장학금
장학금 정보가 궁금해요
학점 계산
서울 중식 메뉴
학식 메뉴 확인할래요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
장학금 정보가 궁금해요
졸업인정제(영어/한자)에 대해서 알려주세요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
학식 메뉴 확인할래요
취업상담센터
졸업하기 위해 필요한 요건이 어떻게 되나요?
출석 어떻게 하나요?
전자출석
졸업하기 위해 필요한 요건이 어떻게 되나요?
졸업하기 위해 필요한 요건이 어떻게 되나요?
참슬기식당
서울캠퍼스 천원의아침밥 먹을래
irims주소
휴학 등록 반환
서울 조식 메뉴
학식 메뉴 확인할래요
휴학에 대해 알려주세요
월별 학사 일정이 궁금해요
중앙사랑장학금
재학생 장학금
장학금정보가 궁금해요
이수 학점 확인하고 싶어요
현재 포탈 사진 보기
초과학기
포탈 사진 변경
스마트학생증 신규발급
학생증/신분증 사진 보기
모바일학생증 안내해주세요
학생증 금융기능
학생증 결제
초과학기 등록금
학생증/신분증 사진 변경 방법 알아보기
학생증 결제
프사 바꾸기
국가 장학금
장학금 정보가 궁금해요
다빈치캠퍼스 천원의아침밥 먹을래
학식 메뉴 확인할래요
서울캠퍼스 셔틀버스 시간표와 노선을 알고 싶어요
셔틀은 무료야?
취업정보센터
근로장학금
재학생 장학금
장학금
중앙사랑장학금
재학생 장학금
장학금 정보가 궁금해요
휴학 환불
CAU세미나
이클래스 강의자료 확인하기
cau세미나 신청방법을 알려주세요
성적우수장학금
재학생 장학금
휴학신청
신입생 장학금
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 중식 메뉴
장학금 정보가 궁금해요
중앙사랑장학금
학식 메뉴 확인할래요
성적우수장학금
재학생 장학금
서울 중식 메뉴
학식 메뉴 확인할래요
이수 학점 확인하고 싶어요
장학금 기준
강의계획서를 확인하고 싶어요
Kmoc
중앙사랑장학금
중앙사랑 신청
강의교재 알려주세요
증명서 발급 방법 알려주세요
강의계획서를 확인하고 싶어요
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
학식 메뉴 확인할래요
서울캠 내일 석식메뉴
서울캠 내일 중식메뉴
서울캠 내일 학식메뉴
서울 석식 메뉴
스마트학생증 재발급
학생증 발급
서울 중식 메뉴
학식 메뉴 확인할래요
중앙사랑장학금
학생증/신분증 발급내역 조회할 수 있나요
중앙사랑
일취월장장학금
부서명
재학생 장학금 안내
역량강화장학금
부서별 위치 및 전화번호
재학생장학금
강의계획서
성적우수장학금
재학생 장학금
201관
장학금 정보가 궁금해요
학생증 재발급
수강신청
이수구분
이수변경
이수 학점 확인하고 싶어요
수강한 과목의 이수 구분을 변경하고 싶어요
학생증/신분증 발급내역 조회할 수 있나요
내 장학 금액이 궁금해
개인장학냐역
강의계획서를 확인하고 싶어요
장학금 이수내역
학자금대출
cau세미나 신청방법을 알려주세요
중앙사랑장학금
중앙사랑
Cau 상담
상담신청
단과대학별 시험일정 어떻게 확인하나요?
중앙사랑장학금
상담 일정 확인하고 싶어요
중앙사랑
    """



In [21]:
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

품사 태깅 10개만 출력 : [('\n', 'Foreign'), ('청강', 'Noun'), ('\n', 'Foreign'), ('청강', 'Noun'), ('신청', 'Noun'), ('\n', 'Foreign'), ('레인보우', 'Noun'), ('시스템', 'Noun'), ('\n', 'Foreign'), ('전과', 'Noun')]
명사 추출 : 청강 청강 신청 레인보우 시스템 전과 수 레인보우 입학 대해 메뉴 설명 서울 캠 내일 중식 메뉴 서울 캠 내일 학식 메뉴 독후감 학점 레인보우 시스템 시스템 학식 메뉴 확인 비대 수업 및 대면 수업 일정 안내 등록금 납부 방법 대해 독후감 학점 마이크로 마이크로 회원 가입 방법 좀 마이크로소프트 이수 학점 확인 수강 정정 대학 규정 및 학칙 대해 서울 캠 내일 중식 메뉴 서울 캠 내일 학식 메뉴 학식 메뉴 확인 대학 규정 및 학칙 대해 재수강 과목 공학 인증 재수강 재수강 전과 학점 학생증 신청 서울 캠 내일 중식 메뉴 서울 캠 내일 학식 메뉴 서울 조식 메뉴 서울 중식 메뉴 학식 메뉴 확인 나 수업 클래스 과제 확인 나 강의 별 과제 시험 점수 클래스 사용 열람 용 성적 증명서 발급 방법 취업 전과 수 재수강 평점 평균 체계 확인 재 입학 방법 이수 학점 확인 예술 대학 교학 지원 팀 연락처 단과대학 교학 지원 팀 연락처 전공 필수 에스컬레이터 학적 등록 졸업 위해 요건 학생증 학적 등록 안 도서관 출입 재발 증명서 발급 방법 예술 대학 교학 지원 팀 연락처 기숙사 부서 클래스 사용 교내 연락처 검색 교내 연락처 검색 이수 학점 확인 학사 일정 서울 캠 내일 중식 메뉴 서울 캠 내일 학식 메뉴 서울 중식 메뉴 학식 메뉴 확인 졸업 정제 영어 한자 대해 졸업 위해 요건 휴학 신청 법 휴학 신청 편입 생 학점 인정 안내 소프트웨어 편입 생 학점 인정 안내 강의 시간표 확인 에스컬레이터 평일 운영 강의 계획 를 확인 수업 폐강 졸업 위해 요건 에스컬레이터 서울 캠 내일 석식 메뉴 서울 캠 내일 학식 메뉴 학식 메

여기서는 사이킷런의 CountVectorizer를 사용하여 단어를 추출합니다.  CountVectorizer를 사용하는 이유는 n_gram_range의 인자를 사용하면 단쉽게 n-gram을 추출할 수 있기 때문입니다. 예를 들어, (2, 3)으로 설정하면 결과 후보는 2개의 단어를 한 묶음으로 간주하는 bigram과 3개의 단어를 한 묶음으로 간주하는 trigram을 추출합니다.

In [22]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5]) # -> 그냥 예시다

trigram 개수 : 12222
trigram 다섯개만 출력 : ['가갹 학생증' '가갹 학생증 재발' '가격 졸업' '가격 졸업 위해' '가격 학식']


다음으로 이제 문서와 문서로부터 추출한 키워드들을 SBERT를 통해서 수치화 할 차례입니다. 한국어를 포함하고 있는 다국어 SBERT를 로드합니다.

In [23]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [24]:
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

이제 문서와 가장 유사한 키워드들을 추출합니다. 여기서는 문서와 가장 유사한 키워드들은 문서를 대표하기 위한 좋은 키워드라고 가정합니다. 상위 5개의 키워드를 출력합니다.

In [25]:
### 20개로 증가 시킴 -> 일단 5개 돌려보고 하기
top_n = 50
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

In [26]:
keywords

['메뉴 단과대학 공지',
 '메뉴 버거 서울',
 '안내 학부 단과대학',
 '메뉴 학생증 분싷',
 '메뉴 점심 서울',
 '메뉴 교직원 교수',
 '메뉴 학식 서울',
 '신청 장학금 서울',
 '메뉴 시발 서울',
 '중식 메뉴 캠퍼스',
 '중식 메뉴 공과대학',
 '학과 사무실 수강',
 '학점 우편물 서울',
 '교과 코드 서울',
 '논문 자격시험 서울',
 '영양학 이메일 서울',
 '장학금 안내 서울',
 '메뉴 대학 교학',
 '수강신청 정정 강의',
 '석식 메뉴 생활관',
 '학생증 신분증 다운로드',
 '시험 신청 서울',
 '지도 교수 서울',
 '메뉴 공과대학 교학',
 '생활관 스터디 학생증',
 '학생증 수강신청 문의',
 '메뉴 학점 서울',
 '학식 메뉴 김효',
 '조식 메뉴 공과대학',
 '메뉴 종합정보시스템 실험실',
 '안내 학생증 공과대학',
 '다운로드 방법 서울',
 '스터디 예약 클래스',
 '메뉴 학식 단과대학',
 '패스트푸드 분식점 클래스',
 '메뉴 설명 서울',
 '메뉴 학생증 발급',
 '교내 장학 신청',
 '안드로이드 단과대학 교학',
 '캠퍼스 석식 메뉴',
 '메뉴 모바일 학생증',
 '튜터 신청 서울',
 '메뉴 학생증 교부',
 '중식 메뉴 수강신청',
 '메뉴 학식 윤석열',
 '메뉴 대학별 교학',
 '캠퍼스 지도 경제학',
 '장학금 신청 서울',
 '교양 강의 수강신청',
 '메뉴 학생증 분실']

5개의 키워드가 출력되는데, 이들의 의미가 좀 비슷해보입니다. 비슷한 의미의 키워드들이 리턴되는 데는 이유가 있습니다. 당연히 이 키워드들이 문서를 가장 잘 나타내고 있기 때문입니다. 만약, 지금 출력한 것보다는 좀 더 다양한 의미의 키워드들이 출력된다면 이들을 그룹으로 본다는 관점에서는 어쩌면 해당 키워드들이 문서를 잘 나타낼 가능성이 적을 수도 있습니다. 따라서 키워드들을 다양하게 출력하고 싶다면 키워드 선정의 정확성과 키워드들의 다양성 사이의 미묘한 균형이 필요합니다.

여기서는 다양한 키워드들을 얻기 위해서 두 가지 알고리즘을 사용합니다.

* Max Sum Similarity  
* Maximal Marginal Relevance

# 2. Max Sum Similarity

데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의됩니다. 여기서의 의도는 후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 것입니다.

In [27]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings,
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

이를 위해 상위 10개의 키워드를 선택하고 이 10개 중에서 서로 가장 유사성이 낮은 5개를 선택합니다.

낮은 nr_candidates를 설정하면 결과는 출력된 키워드 5개는 기존의 코사인 유사도만 사용한 것과 매우 유사한 것으로 보입니다.

In [35]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=7, nr_candidates=25)

['학생증 수강신청 문의',
 '메뉴 종합정보시스템 실험실',
 '다운로드 방법 서울',
 '안드로이드 단과대학 교학',
 '메뉴 모바일 학생증',
 '메뉴 학생증 교부',
 '장학금 신청 서울']

그러나 상대적으로 높은 nr_candidates는 더 다양한 키워드 5개를 만듭니다.

In [36]:
# max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=10, nr_candidates=50)

# 3. Maximal Marginal Relevance

결과를 다양화하는 마지막 방법은 MMR(Maximum Limit Relegance)입니다. MMR은 텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화하기 위해 노력합니다. 참고 할 수 있는 자료로 EmbedRank(https://arxiv.org/pdf/1801.04470.pdf) 라는 키워드 추출 알고리즘은 키워드를 다양화하는 데 사용할 수 있는 MMR을 구현했습니다. 먼저 문서와 가장 유사한 키워드를 선택합니다. 그런 다음 문서와 유사하고 이미 선택된 키워드와 유사하지 않은 새로운 후보를 반복적으로 선택합니다.

In [37]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

만약 우리가 상대적으로 낮은 diversity 값을 설정한다면, 결과는 기존의 코사인 유사도만 사용한 것과 매우 유사한 것으로 보입니다.

In [38]:
# mmr(doc_embedding, candidate_embeddings, candidates, top_n=10, diversity=0.2)

그러나 상대적으로 높은 diversity값은 다양한 키워드 5개를 만들어냅니다.

In [41]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=10, diversity=0.9)

['메뉴 학생증 분실',
 '다빈치 학생',
 '심화 글로벌 금융',
 '성폭력 예방',
 '복수 융합 자기',
 '학교 축구장 동아리',
 '예술 메일',
 '청룡 사회 봉사',
 '야간 수업',
 '기타 장학금']

## topic number = 5개인 경우
1. 꽤 정확하게 나온다 -> 다른 모델도 사용해보고 어떤 것이 나은지 판단하면 될 것 같음
2. 따라서 토픽의 갯수를 늘린 후 다시 실행하는 것으로 진행해보기 : topic의 수를 30~50으로 잡았는데 너무 오래 걸린다. candidate를 높게 하고 그냥 top은 10개정도로 하는게 맞는 것 같다. -> 7/20개는 또 금방나오네? 근데 10/30은 또 오래걸리고
3. 다양성을 늘리려면 MMR를 쓰는게 맞는 것 같다. MSR은 비슷한 내용이 많이 나온다.